In [1]:
import pandas as pd
import numpy as np
import re
import html

In [2]:
df = pd.read_csv('steam_games_enhanced.csv', low_memory=False)

In [3]:
# Normalize the column names
df.columns = df.columns.str.lower().str.replace(' ', '_')

In [4]:
df.head()

,appid,normalized_name,name,release_date,estimated_owners,peak_ccu,required_age,launch_price,price,dlc_count,...,new_owners,new_average_forever,new_median_forever,new_price,new_initialprice,new_discount,new_ccu,new_languages,new_genre,new_tags
0,1026420,warsaw,WARSAW,2019-10-01,20000 - 50000,5,0,"$23,99",23.99,0,...,"20,000 .. 50,000",572.0,496.0,0.0,0.0,0.0,3.0,"English, French, German, Polish, Russian","Indie, RPG, Free To Play","{'Tactical RPG': 255, 'Wargame': 251, 'Turn-Ba..."
1,22670,alienbreed3descent,Alien Breed 3: Descent,2010-11-01,200000 - 500000,3,0,"$9,99",9.99,0,...,"200,000 .. 500,000",161.0,223.0,999.0,999.0,0.0,1.0,"English, French, German, Italian, Japanese, Sp...",Action,"{'Action': 111, 'Shooter': 93, 'Isometric': 89..."
2,231330,deadfalladventures,Deadfall Adventures,2013-11-01,100000 - 200000,4,0,"$19,99",19.99,0,...,"200,000 .. 500,000",671.0,821.0,1999.0,1999.0,0.0,4.0,"English, German, Polish, French, Russian","Action, Adventure","{'Adventure': 143, 'Action': 127, 'FPS': 104, ..."
3,552520,farcry5,Far Cry® 5,2018-03-01,2000000 - 5000000,2164,17,"$59,99",59.99,5,...,"2,000,000 .. 5,000,000",1700.0,1341.0,5999.0,5999.0,0.0,4336.0,"English, French, Italian, German, Spanish - Sp...","Action, Adventure","{'Open World': 1711, 'Action-Adventure': 1707,..."
4,1293830,forzahorizon4,Forza Horizon 4,2021-03-01,2000000 - 5000000,7571,0,"$59,99",59.99,15,...,"5,000,000 .. 10,000,000",2487.0,770.0,0.0,0.0,0.0,6019.0,"English, French, Italian, German, Spanish - Sp...",Racing,"{'Racing': 790, 'Open World': 669, 'Driving': ..."


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5129 entries, 0 to 5128
Data columns (total 69 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   appid                       5129 non-null   int64  
 1   normalized_name             5129 non-null   object 
 2   name                        5129 non-null   object 
 3   release_date                5129 non-null   object 
 4   estimated_owners            5129 non-null   object 
 5   peak_ccu                    5129 non-null   int64  
 6   required_age                5129 non-null   int64  
 7   launch_price                4610 non-null   object 
 8   price                       5129 non-null   float64
 9   dlc_count                   5129 non-null   int64  
 10  about_the_game              5112 non-null   object 
 11  summary                     3051 non-null   object 
 12  supported_languages         5129 non-null   object 
 13  full_audio_languages        5129 

### Data type cleaning

In [6]:
# Transform the 'release_date' column to datetime
df['release_date'] = pd.to_datetime(df['release_date'])

In [7]:
df['estimated_owners'].unique()

array(['20000 - 50000', '200000 - 500000', '100000 - 200000',
       '2000000 - 5000000', '500000 - 1000000', '1000000 - 2000000',
       '50000 - 100000', '0 - 20000', '20000000 - 50000000',
       '5000000 - 10000000', '0 - 0', '10000000 - 20000000',
       '50000000 - 100000000', '100000000 - 200000000'], dtype=object)

In [8]:
df['new_owners'].unique()

array(['20,000 .. 50,000', '200,000 .. 500,000', '2,000,000 .. 5,000,000',
       '5,000,000 .. 10,000,000', '1,000,000 .. 2,000,000',
       '500,000 .. 1,000,000', '100,000 .. 200,000', '0 .. 20,000',
       '50,000 .. 100,000', nan, '20,000,000 .. 50,000,000',
       '50,000,000 .. 100,000,000', '10,000,000 .. 20,000,000',
       '200,000,000 .. 500,000,000', '100,000,000 .. 200,000,000'],
      dtype=object)

In [9]:
df.insert(5, 'min_estimated_owners', df['estimated_owners'].str.split(' - ').str[0].values.astype('int64'))
df.insert(6, 'max_estimated_owners', df['estimated_owners'].str.split(' - ').str[1].values.astype('int64'))

In [10]:
df.drop(columns='estimated_owners', inplace=True)

In [11]:
df.insert(60, 'new_min_estimated_owners', df['new_owners'].str.split(' .. ').str[0].str.replace(',' , '').astype('float'))
df.insert(61, 'new_max_estimated_owners', df['new_owners'].str.split(' .. ').str[1].str.replace(',' , '').astype('float'))

In [12]:
df.drop(columns='new_owners', inplace=True)

In [13]:
(df['max_estimated_owners'] == 0).value_counts()

max_estimated_owners
False    5061
True       68
Name: count, dtype: int64

In [14]:
(df['new_max_estimated_owners'].isna()).value_counts()

new_max_estimated_owners
False    4583
True      546
Name: count, dtype: int64

In [15]:
((df['max_estimated_owners'] == 0) & (df['new_max_estimated_owners'].isna())).value_counts()

False    5061
True       68
Name: count, dtype: int64

In [16]:
df.drop(df[df['max_estimated_owners'] == 0].index, inplace=True)
df.reset_index(drop=True, inplace=True)

In [17]:
(df['new_max_estimated_owners'].isna()).value_counts()

new_max_estimated_owners
False    4583
True      478
Name: count, dtype: int64

In [18]:
df['new_max_estimated_owners'] = df['new_max_estimated_owners'].fillna(df['max_estimated_owners'])

In [19]:
(df['new_max_estimated_owners'].isna()).value_counts()

new_max_estimated_owners
False    5061
Name: count, dtype: int64

In [20]:
(df['new_max_estimated_owners'] < df['max_estimated_owners']).value_counts()

False    4865
True      196
Name: count, dtype: int64

In [21]:
for i, row in df.iterrows():
    df.at[i, 'new_max_estimated_owners'] = max(row['new_max_estimated_owners'], row['max_estimated_owners'])

Now the `max_estimated_owners` is useless and can be dropped

In [22]:
df.drop(columns='max_estimated_owners', inplace=True)

Let's do the same analysis with `min_estiamted_owners` and `new_min_estimated_owners`

In [23]:
(df['min_estimated_owners'] == 0).value_counts()

min_estimated_owners
False    3880
True     1181
Name: count, dtype: int64

In [24]:
(df['new_min_estimated_owners'] == 0).value_counts()

new_min_estimated_owners
False    4414
True      647
Name: count, dtype: int64

In [25]:
df['new_min_estimated_owners'].isna().value_counts()

new_min_estimated_owners
False    4583
True      478
Name: count, dtype: int64

In [26]:
((df['min_estimated_owners'] == 0) & df['new_min_estimated_owners'].isna()).value_counts()

False    4848
True      213
Name: count, dtype: int64

In [27]:
df['new_min_estimated_owners'] = df['new_min_estimated_owners'].fillna(df['min_estimated_owners'])

In [28]:
(df['new_min_estimated_owners'] == 0).value_counts()

new_min_estimated_owners
False    4201
True      860
Name: count, dtype: int64

In [29]:
(df['new_min_estimated_owners'] < df['min_estimated_owners']).value_counts()

False    4865
True      196
Name: count, dtype: int64

In [30]:
for i, row in df.iterrows():
    df.at[i, 'new_min_estimated_owners'] = max(row['new_min_estimated_owners'], row['min_estimated_owners'])

`min_estimated_owners` can now be dropped as completely useless

In [31]:
df.drop(columns='min_estimated_owners', inplace=True)

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5061 entries, 0 to 5060
Data columns (total 69 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   appid                       5061 non-null   int64         
 1   normalized_name             5061 non-null   object        
 2   name                        5061 non-null   object        
 3   release_date                5061 non-null   datetime64[ns]
 4   peak_ccu                    5061 non-null   int64         
 5   required_age                5061 non-null   int64         
 6   launch_price                4583 non-null   object        
 7   price                       5061 non-null   float64       
 8   dlc_count                   5061 non-null   int64         
 9   about_the_game              5045 non-null   object        
 10  summary                     3021 non-null   object        
 11  supported_languages         5061 non-null   object      

In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5061 entries, 0 to 5060
Data columns (total 69 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   appid                       5061 non-null   int64         
 1   normalized_name             5061 non-null   object        
 2   name                        5061 non-null   object        
 3   release_date                5061 non-null   datetime64[ns]
 4   peak_ccu                    5061 non-null   int64         
 5   required_age                5061 non-null   int64         
 6   launch_price                4583 non-null   object        
 7   price                       5061 non-null   float64       
 8   dlc_count                   5061 non-null   int64         
 9   about_the_game              5045 non-null   object        
 10  summary                     3021 non-null   object        
 11  supported_languages         5061 non-null   object      

In [34]:
df['launch_price'] = df['launch_price'].str.replace('\xa0', '', regex=True).str.replace('$' , '').str.replace(',' , '.').values.astype(float)

In [35]:
(df['price'] == 0).value_counts()

price
False    4814
True      247
Name: count, dtype: int64

In [36]:
df['new_price'] = df['new_price']/100
df['new_initialprice'] = df['new_initialprice']/100

In [37]:
((df['price'] != 0) & (df['new_price'].isna())).value_counts()

False    4708
True      353
Name: count, dtype: int64

In [38]:
for i, row in df.iterrows():
    if (row['price'] != 0) & (pd.isna(row['new_price'])):
        df.at[i, 'new_price'] = row['price']

In [39]:
(df['new_price'].isna()).value_counts()

new_price
False    4936
True      125
Name: count, dtype: int64

In [40]:
((df['price'] == 0) & (df['new_price'].isna())).value_counts()

False    4936
True      125
Name: count, dtype: int64

Now the `price` feature is useless:

In [41]:
df.drop(columns='price', inplace=True)

In [42]:
(df['new_initialprice'].isna()).value_counts()

new_initialprice
False    4583
True      478
Name: count, dtype: int64

In [43]:
(df['launch_price'].isna()).value_counts()

launch_price
False    4583
True      478
Name: count, dtype: int64

In [44]:
((df['launch_price'].notna()) & (df['new_initialprice'].isna())).value_counts()

False    5061
Name: count, dtype: int64

In [45]:
(df['launch_price'] - df['new_initialprice'] > 0).value_counts()

False    3898
True     1163
Name: count, dtype: int64

In [46]:
df[['name', 'launch_price', 'new_initialprice']]

,name,launch_price,new_initialprice
0,WARSAW,23.99,0.00
1,Alien Breed 3: Descent,9.99,9.99
2,Deadfall Adventures,19.99,19.99
3,Far Cry® 5,59.99,59.99
4,Forza Horizon 4,59.99,0.00
...,...,...,...
5056,Deathbound,NaN,NaN
5057,F1® Manager 2024,NaN,NaN
5058,SteamWorld Heist II,NaN,NaN
5059,Cat Quest III,NaN,NaN


In [47]:
for i, row in df.iterrows():
    if (row['launch_price'] != 0) & (pd.notna(row['launch_price'])) & (row['new_initialprice'] == 0.):
        df.at[i, 'new_initialprice'] = row['launch_price']

In [48]:
df[['name', 'launch_price', 'new_initialprice']].iloc[0:10]

,name,launch_price,new_initialprice
0,WARSAW,23.99,23.99
1,Alien Breed 3: Descent,9.99,9.99
2,Deadfall Adventures,19.99,19.99
3,Far Cry® 5,59.99,59.99
4,Forza Horizon 4,59.99,59.99
5,Max Payne,9.99,9.99
6,Aurion: Legacy of the Kori-Odan,14.99,4.99
7,Oxygen Not Included,24.99,24.99
8,Puyo Puyo™Tetris®,19.99,19.99
9,DYNASTY WARRIORS 9,59.99,44.99


The column `launch_price` is useless and can be dropped

In [49]:
df.drop(columns='launch_price', inplace=True)

In [50]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5061 entries, 0 to 5060
Data columns (total 67 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   appid                       5061 non-null   int64         
 1   normalized_name             5061 non-null   object        
 2   name                        5061 non-null   object        
 3   release_date                5061 non-null   datetime64[ns]
 4   peak_ccu                    5061 non-null   int64         
 5   required_age                5061 non-null   int64         
 6   dlc_count                   5061 non-null   int64         
 7   about_the_game              5045 non-null   object        
 8   summary                     3021 non-null   object        
 9   supported_languages         5061 non-null   object        
 10  full_audio_languages        5061 non-null   object        
 11  some_reviews                2248 non-null   object      

In [51]:
df.head()

,appid,normalized_name,name,release_date,peak_ccu,required_age,dlc_count,about_the_game,summary,supported_languages,...,new_max_estimated_owners,new_average_forever,new_median_forever,new_price,new_initialprice,new_discount,new_ccu,new_languages,new_genre,new_tags
0,1026420,warsaw,WARSAW,2019-10-01,5,0,0,Use everything at your disposal to help a team...,NaN,"['English', 'French', 'German', 'Polish', 'Rus...",...,50000.0,572.0,496.0,0.00,23.99,0.0,3.0,"English, French, German, Polish, Russian","Indie, RPG, Free To Play","{'Tactical RPG': 255, 'Wargame': 251, 'Turn-Ba..."
1,22670,alienbreed3descent,Alien Breed 3: Descent,2010-11-01,3,0,0,Alien Breed™ 3: Descent is the final explosive...,Alien Breed 3: Descent is the final explosive ...,"['English', 'French', 'German', 'Italian', 'Ja...",...,500000.0,161.0,223.0,9.99,9.99,0.0,1.0,"English, French, German, Italian, Japanese, Sp...",Action,"{'Action': 111, 'Shooter': 93, 'Isometric': 89..."
2,231330,deadfalladventures,Deadfall Adventures,2013-11-01,4,0,0,Deadfall Adventures is an action-driven first-...,NaN,"['English', 'German', 'Polish', 'French', 'Rus...",...,500000.0,671.0,821.0,19.99,19.99,0.0,4.0,"English, German, Polish, French, Russian","Action, Adventure","{'Adventure': 143, 'Action': 127, 'FPS': 104, ..."
3,552520,farcry5,Far Cry® 5,2018-03-01,2164,17,5,Far Cry comes to America in the latest install...,"Welcome to Hope County, Montana, land of the f...","['English', 'French', 'Italian', 'German', 'Sp...",...,5000000.0,1700.0,1341.0,59.99,59.99,0.0,4336.0,"English, French, Italian, German, Spanish - Sp...","Action, Adventure","{'Open World': 1711, 'Action-Adventure': 1707,..."
4,1293830,forzahorizon4,Forza Horizon 4,2021-03-01,7571,0,15,Dynamic seasons change everything at the world...,Dynamic seasons change everything at the world...,"['English', 'French', 'Italian', 'German', 'Sp...",...,10000000.0,2487.0,770.0,0.00,59.99,0.0,6019.0,"English, French, Italian, German, Spanish - Sp...",Racing,"{'Racing': 790, 'Open World': 669, 'Driving': ..."


In [52]:
df['peak_ccu']

0          5
1          3
2          4
3       2164
4       7571
        ... 
5056       0
5057    4578
5058     707
5059     180
5060     216
Name: peak_ccu, Length: 5061, dtype: int64

In [53]:
df['new_ccu']

0          3.0
1          1.0
2          4.0
3       4336.0
4       6019.0
         ...  
5056       NaN
5057       NaN
5058       NaN
5059       NaN
5060       NaN
Name: new_ccu, Length: 5061, dtype: float64

In [54]:
max(0, np.nan)

0

In [55]:
df['new_ccu'].isna().value_counts()

new_ccu
False    4583
True      478
Name: count, dtype: int64

In [56]:
(df['new_ccu'] == 0).value_counts()

new_ccu
False    3966
True     1095
Name: count, dtype: int64

In [57]:
(df['peak_ccu'] == 0).value_counts()

peak_ccu
False    4100
True      961
Name: count, dtype: int64

In [58]:
((df['peak_ccu'] == 0) & (df['new_ccu'].isna())).value_counts()

False    4963
True       98
Name: count, dtype: int64

In [59]:
((df['peak_ccu'] != 0) & (df['new_ccu'].isna())).value_counts()

False    4681
True      380
Name: count, dtype: int64

In [60]:
df['peak_ccu'] = df[['peak_ccu', 'new_ccu']].max(axis=1)

Now the `new_ccu` is useless and can be dropped

In [61]:
df.drop(columns='new_ccu', inplace = True)

In [62]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5061 entries, 0 to 5060
Data columns (total 66 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   appid                       5061 non-null   int64         
 1   normalized_name             5061 non-null   object        
 2   name                        5061 non-null   object        
 3   release_date                5061 non-null   datetime64[ns]
 4   peak_ccu                    5061 non-null   float64       
 5   required_age                5061 non-null   int64         
 6   dlc_count                   5061 non-null   int64         
 7   about_the_game              5045 non-null   object        
 8   summary                     3021 non-null   object        
 9   supported_languages         5061 non-null   object        
 10  full_audio_languages        5061 non-null   object        
 11  some_reviews                2248 non-null   object      

In [63]:
df['supported_languages'] = df['supported_languages'].str.replace('[', '').str.replace(']', '').str.replace("'", '').str.split(', ')

In [64]:
df['full_audio_languages'] = df['full_audio_languages'].str.replace('[', '').str.replace(']', '').str.replace("'", '').str.split(', ')

In [65]:
df['new_languages'] = df['new_languages'].str.split(', ')

In [66]:
df['supported_languages']

0              [English, French, German, Polish, Russian]
1       [English, French, German, Italian, Japanese, S...
2              [English, German, Polish, French, Russian]
3       [English, French, Italian, German, Spanish - S...
4       [English, French, Italian, German, Spanish - S...
                              ...                        
5056    [English, Portuguese - Brazil, French, Italian...
5057    [English, French, Italian, German, Spanish - S...
5058    [English, French, Italian, German, Spanish - S...
5059    [English, French, Italian, German, Spanish - S...
5060    [English, French, German, Spanish - Spain, Jap...
Name: supported_languages, Length: 5061, dtype: object

In [67]:
df['supported_languages'].isna().value_counts()

supported_languages
False    5061
Name: count, dtype: int64

In [68]:
def clean_language_string(text):
    # Decode HTML entities twice
    text = html.unescape(html.unescape(text.strip()))

    # Remove malformed HTML tags
    text = re.sub(r'<.*?>', '', text)

    # Fix broken tag fragments like b/b, B/B, etc., even with spacing
    text = re.sub(r'[bB]\s*/\s*[bB]', '', text)

    # Replace escaped line breaks with commas
    text = text.replace('\\r\\n', ',').replace('\\n', ',').replace('\\r', ',')

    # Remove trailing punctuation and whitespace
    text = re.sub(r'[;:.,\s]+$', '', text)

    return text.strip()

def clean_language_list(lang_list):
    cleaned = []
    for item in lang_list:
        cleaned_str = clean_language_string(item)
        split_parts = [p.strip() for p in cleaned_str.split(',') if p.strip()]
        cleaned.extend(split_parts)
    return list(dict.fromkeys(cleaned))  # Removes duplicates while preserving order

In [69]:
df['supported_languages'] = df['supported_languages'].apply(clean_language_list)

In [70]:
for i, row in df.iterrows():
    if len(row['supported_languages']) == 0:
        print(row['supported_languages'])

In [71]:
df['full_audio_languages'].isna().value_counts()

full_audio_languages
False    5061
Name: count, dtype: int64

In [72]:
df['full_audio_languages'] = df['full_audio_languages'].apply(clean_language_list)

In [73]:
for i, row in df.iterrows():
    if len(row['full_audio_languages']) == 0:
        print(row['full_audio_languages'])

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[

In [74]:
df['new_languages'].isna().value_counts()

new_languages
False    4583
True      478
Name: count, dtype: int64

Since the are no missing values in both the `supported_languages` and the `full_audio_languages` features we can take them as they are and drop the `new_languages` column

In [75]:
df.drop(columns='new_languages', inplace=True)

In [76]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5061 entries, 0 to 5060
Data columns (total 65 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   appid                       5061 non-null   int64         
 1   normalized_name             5061 non-null   object        
 2   name                        5061 non-null   object        
 3   release_date                5061 non-null   datetime64[ns]
 4   peak_ccu                    5061 non-null   float64       
 5   required_age                5061 non-null   int64         
 6   dlc_count                   5061 non-null   int64         
 7   about_the_game              5045 non-null   object        
 8   summary                     3021 non-null   object        
 9   supported_languages         5061 non-null   object        
 10  full_audio_languages        5061 non-null   object        
 11  some_reviews                2248 non-null   object      

It's now time to manage `number_of_reviews`, `reviews_total`, `positive`, `negative`, `new_positive`, `new_negative`

In [77]:
df['number_of_reviews'].isna().value_counts()

number_of_reviews
False    3024
True     2037
Name: count, dtype: int64

In [78]:
(df['number_of_reviews'] == 0).value_counts()

number_of_reviews
False    5061
Name: count, dtype: int64

In [79]:
df['reviews_total'].isna().value_counts()

reviews_total
False    4583
True      478
Name: count, dtype: int64

In [80]:
(df['reviews_total'] == 0).value_counts()

reviews_total
False    5028
True       33
Name: count, dtype: int64

In [81]:
df['positive'].isna().value_counts()

positive
False    5061
Name: count, dtype: int64

In [82]:
(df['positive'] == 0).value_counts()

positive
False    5014
True       47
Name: count, dtype: int64

In [83]:
df['negative'].isna().value_counts()

negative
False    5061
Name: count, dtype: int64

In [84]:
(df['negative'] == 0).value_counts()

negative
False    4963
True       98
Name: count, dtype: int64

In [85]:
df['new_positive'].isna().value_counts()

new_positive
False    4583
True      478
Name: count, dtype: int64

In [86]:
(df['new_positive'] == 0).value_counts()

new_positive
False    5058
True        3
Name: count, dtype: int64

In [87]:
df['new_negative'].isna().value_counts()

new_negative
False    4583
True      478
Name: count, dtype: int64

In [88]:
(df['new_negative'] == 0).value_counts()

new_negative
False    5047
True       14
Name: count, dtype: int64

In [89]:
((df['new_positive'].isna()) & (df['new_negative'].isna())).value_counts()

False    4583
True      478
Name: count, dtype: int64

In [90]:
((df['new_positive'].isna()) & (df['new_negative'].isna()) & (df['reviews_total'].isna())).value_counts()

False    4583
True      478
Name: count, dtype: int64

In [91]:
df['new_positive'] = df['new_positive'].fillna(df['positive'])

In [92]:
(df['new_positive'] == 0).value_counts()

new_positive
False    5027
True       34
Name: count, dtype: int64

In [93]:
df['new_negative'] = df['new_negative'].fillna(df['negative'])

In [94]:
(df['new_negative'] == 0).value_counts()

new_negative
False    5003
True       58
Name: count, dtype: int64

In [95]:
((df['new_positive'] == 0) & (df['new_negative'] == 0)).value_counts()

False    5028
True       33
Name: count, dtype: int64

In [96]:
df[(df['new_positive'] == 0) & (df['new_negative'] == 0)][['name', 'meta_critic_score']]

,name,meta_critic_score
45,Wasteland 2: Director's Cut,80.0
205,F.E.A.R.,88.0
762,ElecHead,84.0
1338,Guild Wars Nightfall®,84.0
1647,SCARF,69.0
2465,RACE On,73.0
2637,Best Month Ever!,65.0
2876,Beijing 2008™ - The Official Video Game of the...,60.0
3213,Guild Wars Factions®,84.0
3566,Lust from Beyond,61.0


This $0$ values needs to be filled

In any case the `positive` and `negative` columns can be dropped.

In [97]:
df.drop(columns=['positive', 'negative'], inplace=True)

In [98]:
df['number_of_reviews']

0       NaN
1         0
2       NaN
3       402
4       223
       ... 
5056      0
5057    NaN
5058    NaN
5059    NaN
5060    NaN
Name: number_of_reviews, Length: 5061, dtype: object

In [99]:
df['number_of_reviews'].isna().value_counts()

number_of_reviews
False    3024
True     2037
Name: count, dtype: int64

In [100]:
def dealing_with_Ks(val):
    if isinstance(val, str):
        val = val.replace(',', '')
        if 'K' in val:
            return float(val.replace('K', '')) * 1000
        return float(val)
    return val  # Keep as-is if not a string

In [101]:
df['number_of_reviews'] = df['number_of_reviews'].apply(dealing_with_Ks)

In [102]:
df['number_of_reviews']

0         NaN
1         0.0
2         NaN
3       402.0
4       223.0
        ...  
5056      0.0
5057      NaN
5058      NaN
5059      NaN
5060      NaN
Name: number_of_reviews, Length: 5061, dtype: float64

In [103]:
df['reviews_total'].isna().value_counts()

reviews_total
False    4583
True      478
Name: count, dtype: int64

In [104]:
df['reviews_total'] = df['reviews_total'].values.astype(float)

In [105]:
((df['reviews_total'].isna()) & (df['number_of_reviews'].isna())).value_counts()

False    4804
True      257
Name: count, dtype: int64

In [106]:
((df['reviews_total'].isna()) & (df['number_of_reviews'].notna()) & (df['number_of_reviews'] != 0)).value_counts()

False    4928
True      133
Name: count, dtype: int64

In [107]:
((df['reviews_total'].isna()) & (df['number_of_reviews'].notna()) & (df['number_of_reviews'] == 0)).value_counts()

False    4973
True       88
Name: count, dtype: int64

In [108]:
((df['reviews_total'] == 0) & (df['number_of_reviews'].notna()) & (df['number_of_reviews'] != 0)).value_counts()

False    5056
True        5
Name: count, dtype: int64

In [109]:
df['reviews_total'] = df['reviews_total'].fillna(df['number_of_reviews'])

In [110]:
for i, row in df.iterrows():
    if (row['reviews_total'] == 0) & (pd.notna(row['number_of_reviews'])) & (row['number_of_reviews'] != 0):
        df.at[i, 'reviews_total'] = row['number_of_reviews']

In [111]:
df['reviews_total'].isna().value_counts()

reviews_total
False    4804
True      257
Name: count, dtype: int64

Let's drop the number of reviews column

In [112]:
df.drop(columns='number_of_reviews', inplace=True)

Now in the `reviews_total` column there are both $0$ and NaN. We have to deal with that.

In [113]:
((df['reviews_total'].isna()) & ((df['new_positive'] + df['new_negative']) != 0)).value_counts()

False    4825
True      236
Name: count, dtype: int64

In [114]:
((df['reviews_total'] == 0) & ((df['new_positive'] + df['new_negative']) != 0)).value_counts()

False    4948
True      113
Name: count, dtype: int64

In [115]:
((df['reviews_total'].isna()) & ((df['new_positive'] + df['new_negative']) == 0)).value_counts()

False    5040
True       21
Name: count, dtype: int64

In [116]:
((df['reviews_total'] == 0) & ((df['new_positive'] + df['new_negative']) == 0)).value_counts()

False    5058
True        3
Name: count, dtype: int64

In [117]:
df['reviews_total'] = df['reviews_total'].fillna(df['new_positive'] + df['new_negative'])

In [118]:
for i, row in df.iterrows():
    if (row['reviews_total'] == 0) & ((row['new_positive'] + row['new_negative']) != 0):
        df.at[i, 'reviews_total'] = row['new_positive'] + row['new_negative']

In [119]:
(df['reviews_total'] < df['new_positive'] + df['new_negative']).value_counts()

True     4661
False     400
Name: count, dtype: int64

In [120]:
for i, row in df.iterrows():
    if row['reviews_total'] < row['new_positive'] + row['new_negative']:
        df.at[i, 'reviews_total'] = row['new_positive'] + row['new_negative']

In [121]:
df['reviews_total'].isna().value_counts()

reviews_total
False    5061
Name: count, dtype: int64

In [122]:
(df['reviews_total'] == 0).value_counts()

reviews_total
False    5037
True       24
Name: count, dtype: int64

In [123]:
df['reviews_score_fancy'] = df['reviews_score_fancy'].str.replace('%' , '').str.replace(',' , '.').values.astype(float)

In [124]:
df['reviews_score_fancy'].isna().value_counts()

reviews_score_fancy
False    4583
True      478
Name: count, dtype: int64

In [125]:
(df['new_positive'] + df['new_negative']).isna().value_counts()

False    5061
Name: count, dtype: int64

In [126]:
(df['new_positive'] + df['new_negative'] == 0).value_counts()

False    5028
True       33
Name: count, dtype: int64

In [127]:
(100*df['new_positive']*1/(df['new_positive'] + df['new_negative'])).isna().value_counts()

False    5028
True       33
Name: count, dtype: int64

In [128]:
((df['reviews_score_fancy'].notna()) & (round(100*df['new_positive']*1/(df['new_positive'] + df['new_negative']))).isna()).value_counts()

False    5058
True        3
Name: count, dtype: int64

In [129]:
round(100*df['new_positive']*1/(df['new_positive'] + df['new_negative']))

0       65.0
1       75.0
2       74.0
3       81.0
4       91.0
        ... 
5056    75.0
5057    66.0
5058    96.0
5059    95.0
5060    97.0
Length: 5061, dtype: float64

In [130]:
df['new_reviews_score_fancy'] = round(100*df['new_positive']*1/(df['new_positive'] + df['new_negative']))

In [131]:
df['new_reviews_score_fancy'] = df['new_reviews_score_fancy'].fillna('reviews_score_fancy')

In [132]:
df['reviews_score_fancy'] = df['new_reviews_score_fancy']

In [133]:
df['reviews_score_fancy']

0       65.0
1       75.0
2       74.0
3       81.0
4       91.0
        ... 
5056    75.0
5057    66.0
5058    96.0
5059    95.0
5060    97.0
Name: reviews_score_fancy, Length: 5061, dtype: object

The column `new_reviews_score_fancy` can now be dropped

In [134]:
df.drop(columns='new_reviews_score_fancy', inplace=True)

In [135]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5061 entries, 0 to 5060
Data columns (total 62 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   appid                       5061 non-null   int64         
 1   normalized_name             5061 non-null   object        
 2   name                        5061 non-null   object        
 3   release_date                5061 non-null   datetime64[ns]
 4   peak_ccu                    5061 non-null   float64       
 5   required_age                5061 non-null   int64         
 6   dlc_count                   5061 non-null   int64         
 7   about_the_game              5045 non-null   object        
 8   summary                     3021 non-null   object        
 9   supported_languages         5061 non-null   object        
 10  full_audio_languages        5061 non-null   object        
 11  some_reviews                2248 non-null   object      

The columns `website`, `support_url` and `support_email` are quite useless and can be dropped

In [136]:
df.drop(columns=['website','support_url', 'support_email'], inplace=True)

In [137]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5061 entries, 0 to 5060
Data columns (total 59 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   appid                       5061 non-null   int64         
 1   normalized_name             5061 non-null   object        
 2   name                        5061 non-null   object        
 3   release_date                5061 non-null   datetime64[ns]
 4   peak_ccu                    5061 non-null   float64       
 5   required_age                5061 non-null   int64         
 6   dlc_count                   5061 non-null   int64         
 7   about_the_game              5045 non-null   object        
 8   summary                     3021 non-null   object        
 9   supported_languages         5061 non-null   object        
 10  full_audio_languages        5061 non-null   object        
 11  some_reviews                2248 non-null   object      

In [138]:
df['header_image'].values

array(['https://cdn.akamai.steamstatic.com/steam/apps/1026420/header.jpg?t=1657716289',
       'https://cdn.akamai.steamstatic.com/steam/apps/22670/header.jpg?t=1655730869',
       'https://cdn.akamai.steamstatic.com/steam/apps/231330/header.jpg?t=1561964976',
       ...,
       'https://shared.akamai.steamstatic.com/store_item_assets/steam/apps/2396240/header.jpg?t=1723132341',
       'https://shared.akamai.steamstatic.com/store_item_assets/steam/apps/2305840/header.jpg?t=1723195692',
       'https://shared.akamai.steamstatic.com/store_item_assets/steam/apps/2366980/header.jpg?t=1722529446'],
      dtype=object)

In [139]:
(df['metacritic_score'] == 0).value_counts()

metacritic_score
False    2736
True     2325
Name: count, dtype: int64

Since we have added the `meta_critic_score` column the `metacritic_score` one can be dropped

In [140]:
df.drop(columns='metacritic_score', inplace=True)

In [141]:
df['metacritic_url']

0       https://www.metacritic.com/game/pc/warsaw?ftag...
1       https://www.metacritic.com/game/pc/alien-breed...
2       https://www.metacritic.com/game/pc/deadfall-ad...
3                                                     NaN
4                                                     NaN
                              ...                        
5056                                                  NaN
5057                                                  NaN
5058    https://www.metacritic.com/game/pc/steamworld-...
5059    https://www.metacritic.com/game/pc/cat-quest-i...
5060    https://www.metacritic.com/game/pc/thank-goodn...
Name: metacritic_url, Length: 5061, dtype: object

In [142]:
import re

# Function to normalize game names to the metacritic standard
def metacritic_name(name):
    # Convert to lowercase
    name = name.lower()
    # Remove trademarks (e.g., ™, ®)
    name = re.sub(r'[™®]', '', name)
    # Remove other symbols (e.g., punctuation)
    name = re.sub(r'[^\w\s]', '', name)
    # Replace spaces with '-'
    name = name.replace(' ', '-')
    return name

In [143]:
metacritic_url_list = []
for i, row in df.iterrows():
    if pd.isna(row['metacritic_url']):
        metacritic_url_list.append('https://www.metacritic.com/game/pc/' + metacritic_name(row['name']) + '?ftag=MCD-06-10aaa1f')
    else:
        metacritic_url_list.append(row['metacritic_url'])

In [144]:
df['metacritic_url'] = metacritic_url_list

In [145]:
df['metacritic_url'].values

array(['https://www.metacritic.com/game/pc/warsaw?ftag=MCD-06-10aaa1f',
       'https://www.metacritic.com/game/pc/alien-breed-3-descent?ftag=MCD-06-10aaa1f',
       'https://www.metacritic.com/game/pc/deadfall-adventures?ftag=MCD-06-10aaa1f',
       ...,
       'https://www.metacritic.com/game/pc/steamworld-heist-ii?ftag=MCD-06-10aaa1f',
       'https://www.metacritic.com/game/pc/cat-quest-iii?ftag=MCD-06-10aaa1f',
       'https://www.metacritic.com/game/pc/thank-goodness-youre-here!?ftag=MCD-06-10aaa1f'],
      dtype=object)

In [146]:
df['metacritic_url'].isna().value_counts()

metacritic_url
False    5061
Name: count, dtype: int64

In [147]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5061 entries, 0 to 5060
Data columns (total 58 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   appid                       5061 non-null   int64         
 1   normalized_name             5061 non-null   object        
 2   name                        5061 non-null   object        
 3   release_date                5061 non-null   datetime64[ns]
 4   peak_ccu                    5061 non-null   float64       
 5   required_age                5061 non-null   int64         
 6   dlc_count                   5061 non-null   int64         
 7   about_the_game              5045 non-null   object        
 8   summary                     3021 non-null   object        
 9   supported_languages         5061 non-null   object        
 10  full_audio_languages        5061 non-null   object        
 11  some_reviews                2248 non-null   object      

In [148]:
(df['user_score'] == 0).value_counts()

user_score
True     5060
False       1
Name: count, dtype: int64

The columns `user_score` and `score_rank` can both be dropped since they have only 1 non null value

In [149]:
df.drop(columns=['user_score', 'score_rank'], inplace=True)

In [150]:
metacritic_user_score = pd.read_csv('user_score_updated.csv')
df = df.merge(metacritic_user_score, left_on='appid', right_on='appid', how='left')

In [151]:
df.rename(columns={'user_score': 'metacritic_user_score'}, inplace=True)

In [152]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5061 entries, 0 to 5060
Data columns (total 57 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   appid                       5061 non-null   int64         
 1   normalized_name             5061 non-null   object        
 2   name                        5061 non-null   object        
 3   release_date                5061 non-null   datetime64[ns]
 4   peak_ccu                    5061 non-null   float64       
 5   required_age                5061 non-null   int64         
 6   dlc_count                   5061 non-null   int64         
 7   about_the_game              5045 non-null   object        
 8   summary                     3021 non-null   object        
 9   supported_languages         5061 non-null   object        
 10  full_audio_languages        5061 non-null   object        
 11  some_reviews                2248 non-null   object      

In [153]:
df[(df['meta_critic_score'].notna()) & (df['metacritic_user_score'] == '-1')][['appid', 'name', 'meta_critic_score', 'metacritic_user_score']]

,appid,name,meta_critic_score,metacritic_user_score
91,378750,Sayonara Umihara Kawase,76.0,-1
786,236450,PAC-MAN™ Championship Edition DX+,93.0,-1
970,331750,Disney•Pixar WALL-E,50.0,-1
3479,469820,Genital Jousting,68.0,-1
3566,1035120,Lust from Beyond,61.0,-1


In [154]:
df.at[91, 'metacritic_user_score'] = '8.3'

In [155]:
df.at[786, 'metacritic_user_score'] = '7.6'

In [156]:
df.at[970, 'metacritic_user_score'] = '6.6'

In [157]:
df.at[3479, 'metacritic_user_score'] = '5.3'

In [158]:
df[(df['meta_critic_score'].notna()) & (df['metacritic_user_score'] == 'tbd')][['appid', 'name', 'meta_critic_score', 'metacritic_user_score']]

,appid,name,meta_critic_score,metacritic_user_score
25,1495860,What The Dub?!,78.0,tbd
27,16130,Fish Tycoon,46.0,tbd
34,1095120,Helheim Hassle,79.0,tbd
37,1228870,Bartlow's Dread Machine,73.0,tbd
56,1275660,Sanity of Morris,35.0,tbd
...,...,...,...,...
4981,2104880,Undead Inc.,59.0,tbd
4984,2739830,King Arthur: Legion IX,72.0,tbd
4990,1293910,Read Only Memories: NEURODIVER,77.0,tbd
4997,2719200,SAMURAI WARRIORS 4 DX,82.0,tbd


In [159]:
for idx, row in df.iterrows():
    if (row['metacritic_user_score'] == '-1') or (row['metacritic_user_score'] == 'tbd'):
        df.at[idx, 'metacritic_user_score'] = np.nan

df['metacritic_user_score'] = df['metacritic_user_score'].astype(float)

In [160]:
df['metacritic_user_score']

0       7.1
1       6.7
2       6.8
3       6.9
4       8.4
       ... 
5056    6.6
5057    7.5
5058    7.6
5059    8.1
5060    7.7
Name: metacritic_user_score, Length: 5061, dtype: float64

In [161]:
df['rating'].isna().value_counts()

rating
False    2768
True     2293
Name: count, dtype: int64

In [162]:
(df['rating'] == 0).value_counts()

rating
False    5061
Name: count, dtype: int64

In [163]:
((df['rating'].notna()) & (df['reviews_score_fancy'] == 0)).value_counts()

False    5061
Name: count, dtype: int64

The column rating is completely useless and can be dropped

In [164]:
df.drop(columns='rating', inplace=True)

In [165]:
df['achievements']

0        34
1        13
2        50
3         0
4       178
       ... 
5056     35
5057     30
5058     34
5059     30
5060     32
Name: achievements, Length: 5061, dtype: int64

In [166]:
(df['achievements'] == 0).value_counts()

achievements
False    4182
True      879
Name: count, dtype: int64

In [167]:
df['recommendations']

0          427
1          285
2         1140
3       114588
4       126316
         ...  
5056         0
5057      1205
5058       469
5059      1294
5060      2222
Name: recommendations, Length: 5061, dtype: int64

In [168]:
(df['recommendations'] == 0).value_counts()

recommendations
False    4129
True      932
Name: count, dtype: int64

In [169]:
df[df['notes'].notna()]['notes']

15      This game contains content that might not be s...
19      This product features frequent violence and ba...
35      Phoenix Point contains depictions of science-f...
36      This Game may contain content not appropriate ...
56      Sanity of Morris is a horror adventure game. S...
                              ...                        
5048    The game talks about death and loss, while not...
5049    The game contains some video depicting murders...
5050                     Volgarr kills evil in all forms!
5055    Light sexual themes and occasional references ...
5056                          Excessive Blood Dead Bodies
Name: notes, Length: 591, dtype: object

`average_playtime_two_weeks`, `median_playtime_two_weeks` are not relevant and can be dropped

In [170]:
df.drop(columns=['average_playtime_two_weeks', 'median_playtime_two_weeks'], inplace=True)

In [171]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5061 entries, 0 to 5060
Data columns (total 54 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   appid                     5061 non-null   int64         
 1   normalized_name           5061 non-null   object        
 2   name                      5061 non-null   object        
 3   release_date              5061 non-null   datetime64[ns]
 4   peak_ccu                  5061 non-null   float64       
 5   required_age              5061 non-null   int64         
 6   dlc_count                 5061 non-null   int64         
 7   about_the_game            5045 non-null   object        
 8   summary                   3021 non-null   object        
 9   supported_languages       5061 non-null   object        
 10  full_audio_languages      5061 non-null   object        
 11  some_reviews              2248 non-null   object        
 12  reviews_total       

In [172]:
(df['average_playtime_forever'] == 0).value_counts()

average_playtime_forever
False    3710
True     1351
Name: count, dtype: int64

In [173]:
df['new_average_forever'].isna().value_counts()

new_average_forever
False    4583
True      478
Name: count, dtype: int64

In [174]:
(df['new_average_forever'] == 0).value_counts()

new_average_forever
False    4072
True      989
Name: count, dtype: int64

In [175]:
df[['average_playtime_forever', 'new_average_forever']]

,average_playtime_forever,new_average_forever
0,67,572.0
1,44,161.0
2,324,671.0
3,1756,1700.0
4,2518,2487.0
...,...,...
5056,0,NaN
5057,0,NaN
5058,50,NaN
5059,0,NaN


In [176]:
((df['average_playtime_forever'] != 0) & (df['new_average_forever'].isna())).value_counts()

False    4868
True      193
Name: count, dtype: int64

In [177]:
((df['average_playtime_forever'] != 0) & (df['new_average_forever'] == 0)).value_counts()

False    4728
True      333
Name: count, dtype: int64

In [178]:
df['new_average_forever'] = df['new_average_forever'].fillna(df['average_playtime_forever'])

In [179]:
((df['average_playtime_forever'] != 0) & (df['new_average_forever'] == 0)).value_counts()

False    4728
True      333
Name: count, dtype: int64

In [180]:
for i, row in df.iterrows():
    if (row['average_playtime_forever'] != 0) & (row['new_average_forever'] == 0):
        df.at[i, 'new_average_forever'] = row['average_playtime_forever']

In [181]:
(df['new_average_forever'] == 0).value_counts()

new_average_forever
False    4120
True      941
Name: count, dtype: int64

In [182]:
df[['average_playtime_forever', 'new_average_forever']]

,average_playtime_forever,new_average_forever
0,67,572.0
1,44,161.0
2,324,671.0
3,1756,1700.0
4,2518,2487.0
...,...,...
5056,0,0.0
5057,0,0.0
5058,50,50.0
5059,0,0.0


`average_playtime_forever` is now useless and can be dropped

In [183]:
df.drop(columns='average_playtime_forever', inplace=True)

The same type of analysis now needs to be done on `median_playtime_forever` and `new_median_forever`

In [184]:
(df['median_playtime_forever'] == 0).value_counts()

median_playtime_forever
False    3710
True     1351
Name: count, dtype: int64

In [185]:
df['new_median_forever'].isna().value_counts()

new_median_forever
False    4583
True      478
Name: count, dtype: int64

In [186]:
df[['median_playtime_forever', 'new_median_forever']]

,median_playtime_forever,new_median_forever
0,93,496.0
1,24,223.0
2,431,821.0
3,1258,1341.0
4,1021,770.0
...,...,...
5056,0,NaN
5057,0,NaN
5058,50,NaN
5059,0,NaN


In [187]:
((df['median_playtime_forever'] != 0) & (df['new_median_forever'].isna())).value_counts()

False    4868
True      193
Name: count, dtype: int64

In [188]:
((df['median_playtime_forever'] != 0) & (df['new_median_forever'] == 0)).value_counts()

False    4728
True      333
Name: count, dtype: int64

In [189]:
df['new_median_forever'] = df['new_median_forever'].fillna(df['median_playtime_forever'])

In [190]:
for i, row in df.iterrows():
    if (row['median_playtime_forever'] != 0) & (row['new_median_forever'] == 0):
        df.at[i, 'new_median_forever'] = row['median_playtime_forever']

In [191]:
(df['new_median_forever'] == 0).value_counts()

new_median_forever
False    4120
True      941
Name: count, dtype: int64

In [192]:
df[['median_playtime_forever', 'new_median_forever']]

,median_playtime_forever,new_median_forever
0,93,496.0
1,24,223.0
2,431,821.0
3,1258,1341.0
4,1021,770.0
...,...,...
5056,0,0.0
5057,0,0.0
5058,50,50.0
5059,0,0.0


`median_playtime_forever` is now useless and can be dropped

In [193]:
df.drop(columns='median_playtime_forever', inplace=True)

In [194]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5061 entries, 0 to 5060
Data columns (total 52 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   appid                     5061 non-null   int64         
 1   normalized_name           5061 non-null   object        
 2   name                      5061 non-null   object        
 3   release_date              5061 non-null   datetime64[ns]
 4   peak_ccu                  5061 non-null   float64       
 5   required_age              5061 non-null   int64         
 6   dlc_count                 5061 non-null   int64         
 7   about_the_game            5045 non-null   object        
 8   summary                   3021 non-null   object        
 9   supported_languages       5061 non-null   object        
 10  full_audio_languages      5061 non-null   object        
 11  some_reviews              2248 non-null   object        
 12  reviews_total       

In [195]:
df['developers']

0                                          Pixelated Milk
1                                      Team17 Digital Ltd
2                                             The Farm 51
3       Ubisoft Montreal,Red Storm,Ubisoft Shanghai,Ub...
4                                        Playground Games
                              ...                        
5056                                    Trialforge Studio
5057                                Frontier Developments
5058                               Thunderful Development
5059                                       The Gentlebros
5060                                          Coal Supper
Name: developers, Length: 5061, dtype: object

In [196]:
df['developers'] = df['developers'].str.split(',')

In [197]:
def merge_suffixes(companies):
    # Normalize suffixes (no leading/trailing spaces)
    suffixes = {'Inc.', 'Ltd.', 'LTD.'}
    result = []

    for item in companies:
        stripped = item.strip()  # Remove leading/trailing spaces
        if stripped in suffixes and result:
            # Merge with previous, preserving original spacing
            result[-1] += ' ' + stripped
        else:
            result.append(item.strip())

    return result

In [198]:
df['developers'] = df['developers'].apply(merge_suffixes)

In [199]:
df['publishers']

0       Pixelated Milk,gaming company
1                  Team17 Digital Ltd
2                          THQ Nordic
3                             Ubisoft
4                   Xbox Game Studios
                    ...              
5056                  Tate Multimedia
5057            Frontier Developments
5058            Thunderful Publishing
5059               Kepler Interactive
5060                            Panic
Name: publishers, Length: 5061, dtype: object

In [200]:
df['publishers'] = df['publishers'].str.split(',')

In [201]:
for idx, row in df.iterrows():
    if not isinstance(row['publishers'], float):
        df.at[idx, 'publishers'] = merge_suffixes(row['publishers'])

In [202]:
df['publishers']

0       [Pixelated Milk, gaming company]
1                   [Team17 Digital Ltd]
2                           [THQ Nordic]
3                              [Ubisoft]
4                    [Xbox Game Studios]
                      ...               
5056                   [Tate Multimedia]
5057             [Frontier Developments]
5058             [Thunderful Publishing]
5059                [Kepler Interactive]
5060                             [Panic]
Name: publishers, Length: 5061, dtype: object

In [203]:
df['categories']

0       Single-player,Steam Achievements,Steam Trading...
1       Single-player,Multi-player,Co-op,Steam Achieve...
2       Single-player,Multi-player,Co-op,Steam Achieve...
3       Single-player,Multi-player,Co-op,Online Co-op,...
4       Single-player,Multi-player,PvP,Online PvP,Co-o...
                              ...                        
5056    Single-player,Steam Achievements,Full controll...
5057    Single-player,Steam Achievements,Steam Trading...
5058    Single-player,Steam Achievements,Full controll...
5059    Single-player,Multi-player,Co-op,Shared/Split ...
5060    Single-player,Steam Achievements,Full controll...
Name: categories, Length: 5061, dtype: object

In [204]:
df['categories'] = df['categories'].str.split(',').values

In [205]:
df['categories']

0       [Single-player, Steam Achievements, Steam Trad...
1       [Single-player, Multi-player, Co-op, Steam Ach...
2       [Single-player, Multi-player, Co-op, Steam Ach...
3       [Single-player, Multi-player, Co-op, Online Co...
4       [Single-player, Multi-player, PvP, Online PvP,...
                              ...                        
5056    [Single-player, Steam Achievements, Full contr...
5057    [Single-player, Steam Achievements, Steam Trad...
5058    [Single-player, Steam Achievements, Full contr...
5059    [Single-player, Multi-player, Co-op, Shared/Sp...
5060    [Single-player, Steam Achievements, Full contr...
Name: categories, Length: 5061, dtype: object

It's now the moment to compare `genres` and `tags` with `new_genre` and `new_tags`

In [206]:
df['genres']

0                                 Indie,RPG
1                                    Action
2                          Action,Adventure
3                          Action,Adventure
4                                    Racing
                       ...                 
5056             Action,Adventure,Indie,RPG
5057                    Simulation,Strategy
5058    Action,Adventure,Indie,RPG,Strategy
5059             Action,Adventure,Indie,RPG
5060                 Adventure,Casual,Indie
Name: genres, Length: 5061, dtype: object

In [207]:
df['genres'] = df['genres'].str.split(',')

In [208]:
df['genres']

0                                    [Indie, RPG]
1                                        [Action]
2                             [Action, Adventure]
3                             [Action, Adventure]
4                                        [Racing]
                          ...                    
5056              [Action, Adventure, Indie, RPG]
5057                       [Simulation, Strategy]
5058    [Action, Adventure, Indie, RPG, Strategy]
5059              [Action, Adventure, Indie, RPG]
5060                   [Adventure, Casual, Indie]
Name: genres, Length: 5061, dtype: object

In [209]:
df['new_genre']

0       Indie, RPG, Free To Play
1                         Action
2              Action, Adventure
3              Action, Adventure
4                         Racing
                  ...           
5056                         NaN
5057                         NaN
5058                         NaN
5059                         NaN
5060                         NaN
Name: new_genre, Length: 5061, dtype: object

In [210]:
df['new_genre'] = df['new_genre'].str.split(', ')

In [211]:
df['new_genre']

0       [Indie, RPG, Free To Play]
1                         [Action]
2              [Action, Adventure]
3              [Action, Adventure]
4                         [Racing]
                   ...            
5056                           NaN
5057                           NaN
5058                           NaN
5059                           NaN
5060                           NaN
Name: new_genre, Length: 5061, dtype: object

In [212]:
df.iloc[10]

appid                                                                 1345820
normalized_name                                                     ragnarock
name                                                                Ragnarock
release_date                                              2021-07-01 00:00:00
peak_ccu                                                                 44.0
required_age                                                                0
dlc_count                                                                  17
about_the_game              Are you ready to rock? Ragnarock is a solo/mul...
summary                     Ragnarock is a VR game of rhythm and skill. Yo...
supported_languages         [English, French, Italian, German, Spanish - S...
full_audio_languages                                                       []
some_reviews                “You know that scene in 'Mad Max: Fury Road' w...
reviews_total                                                   

In [213]:
for genres, new_genres in zip(df['genres'].values, df['new_genre'].values):
    if isinstance(genres, list) and isinstance(new_genres, float):
        print(genres, new_genres)

['Adventure', 'Indie', 'RPG'] nan
['Action', 'Adventure', 'Free to Play'] nan
['Adventure', 'Indie', 'RPG', 'Strategy'] nan
['Adventure', 'Indie'] nan
['Adventure'] nan
['Free to Play', 'Massively Multiplayer', 'RPG'] nan
['Simulation', 'Sports'] nan
['Massively Multiplayer', 'RPG'] nan
['Action'] nan
['Action'] nan
['Action'] nan
['Action'] nan
['Action', 'Free to Play'] nan
['Action', 'Indie', 'Strategy'] nan
['Action', 'Adventure', 'Casual', 'Indie'] nan
['Action', 'Adventure', 'Indie'] nan
['Action', 'Free to Play'] nan
['Simulation', 'Sports'] nan
['Adventure', 'Massively Multiplayer', 'RPG', 'Strategy'] nan
['RPG'] nan
['Action'] nan
['Casual', 'Indie', 'Strategy'] nan
['Free to Play', 'Massively Multiplayer', 'RPG'] nan
['Indie', 'Strategy'] nan
['Adventure'] nan
['Action', 'RPG'] nan
['Action'] nan
['RPG'] nan
['Adventure'] nan
['Action', 'Free to Play', 'Massively Multiplayer', 'Simulation'] nan
['Action', 'Adventure'] nan
['Action', 'Free to Play', 'Massively Multiplayer', 'R

In [214]:
for genres, new_genres in zip(df['genres'].values, df['new_genre'].values):
    if isinstance(genres, float) and isinstance(new_genres, list):
        print(genres, new_genres)

The above cell tell us that the missing values in the `genres` column doesn't correspond to non null values in `new_genre` that is now quite useless and can be dropped

In [215]:
df.drop(columns='new_genre', inplace = True)

In [216]:
df['tags']

0       Tactical RPG,Turn-Based Strategy,Wargame,Histo...
1       Action,Shooter,Sci-fi,Aliens,Third Person,Isom...
2       Adventure,Action,FPS,Shooter,Multiplayer,First...
3       Open World,Co-op,Action,Multiplayer,FPS,Shoote...
4       Racing,Open World,Driving,Multiplayer,Online C...
                              ...                        
5056    Souls-like,RPG,Difficult,Action RPG,Violent,St...
5057    Simulation,Strategy,Racing,Automobile Sim,Imme...
5058    Comedy,Steampunk,Turn-Based Combat,Strategy,Tu...
5059    Action,Adventure,Cats,Action RPG,Exploration,L...
5060    Adventure,Comedy,Hand-drawn,Funny,Exploration,...
Name: tags, Length: 5061, dtype: object

In [217]:
df['tags'] = df['tags'].str.split(',')

In [218]:
df['tags']

0       [Tactical RPG, Turn-Based Strategy, Wargame, H...
1       [Action, Shooter, Sci-fi, Aliens, Third Person...
2       [Adventure, Action, FPS, Shooter, Multiplayer,...
3       [Open World, Co-op, Action, Multiplayer, FPS, ...
4       [Racing, Open World, Driving, Multiplayer, Onl...
                              ...                        
5056    [Souls-like, RPG, Difficult, Action RPG, Viole...
5057    [Simulation, Strategy, Racing, Automobile Sim,...
5058    [Comedy, Steampunk, Turn-Based Combat, Strateg...
5059    [Action, Adventure, Cats, Action RPG, Explorat...
5060    [Adventure, Comedy, Hand-drawn, Funny, Explora...
Name: tags, Length: 5061, dtype: object

In [219]:
df['new_tags']

0       {'Tactical RPG': 255, 'Wargame': 251, 'Turn-Ba...
1       {'Action': 111, 'Shooter': 93, 'Isometric': 89...
2       {'Adventure': 143, 'Action': 127, 'FPS': 104, ...
3       {'Open World': 1711, 'Action-Adventure': 1707,...
4       {'Racing': 790, 'Open World': 669, 'Driving': ...
                              ...                        
5056                                                  NaN
5057                                                  NaN
5058                                                  NaN
5059                                                  NaN
5060                                                  NaN
Name: new_tags, Length: 5061, dtype: object

In [220]:
for i, row in df.iterrows():
    if isinstance(row['new_tags'], str):
       if isinstance(eval(row['new_tags']), list):
           print(eval(row['new_tags']))

[]


In [221]:
new_tags_list = []
for i, row in df.iterrows():
    if isinstance(row['new_tags'], str):
        if isinstance(eval(row['new_tags']), dict):
            new_tags_list.append(eval(row['new_tags']).keys())
        else:
            df.at[i, 'new_tags'] = np.nan
            new_tags_list.append(row['new_tags'])
    else:
        new_tags_list.append(row['new_tags'])

for i, elem in enumerate(new_tags_list):
    if not isinstance(elem, float):
        new_tags_list[i] = list(elem)

In [222]:
df['new_tags'] = new_tags_list

In [223]:
df['new_tags']

0       [Tactical RPG, Wargame, Turn-Based Strategy, H...
1       [Action, Shooter, Isometric, Aliens, Sci-fi, T...
2       [Adventure, Action, FPS, Shooter, First-Person...
3       [Open World, Action-Adventure, Multiplayer, Co...
4       [Racing, Open World, Driving, Multiplayer, Aut...
                              ...                        
5056                                                  NaN
5057                                                  NaN
5058                                                  NaN
5059                                                  NaN
5060                                                  NaN
Name: new_tags, Length: 5061, dtype: object

In [224]:
df[['tags', 'new_tags']]

,tags,new_tags
0,"[Tactical RPG, Turn-Based Strategy, Wargame, H...","[Tactical RPG, Wargame, Turn-Based Strategy, H..."
1,"[Action, Shooter, Sci-fi, Aliens, Third Person...","[Action, Shooter, Isometric, Aliens, Sci-fi, T..."
2,"[Adventure, Action, FPS, Shooter, Multiplayer,...","[Adventure, Action, FPS, Shooter, First-Person..."
3,"[Open World, Co-op, Action, Multiplayer, FPS, ...","[Open World, Action-Adventure, Multiplayer, Co..."
4,"[Racing, Open World, Driving, Multiplayer, Onl...","[Racing, Open World, Driving, Multiplayer, Aut..."
...,...,...
5056,"[Souls-like, RPG, Difficult, Action RPG, Viole...",NaN
5057,"[Simulation, Strategy, Racing, Automobile Sim,...",NaN
5058,"[Comedy, Steampunk, Turn-Based Combat, Strateg...",NaN
5059,"[Action, Adventure, Cats, Action RPG, Explorat...",NaN


In [225]:
for tags, new_tags in zip(df['tags'].values, df['new_tags'].values):
    if isinstance(tags, float) and isinstance(new_tags, list):
        print(tags, new_tags)

nan ['Casual', 'Arcade', 'Puzzle-Platformer', '2D', 'Score Attack', 'Singleplayer', 'Indie']
nan ['Adventure', 'Action', 'Strategy', 'Turn-Based Tactics', '4X', 'Arcade', 'Grand Strategy', 'Turn-Based Strategy', 'Visual Novel', 'Wargame', 'Tactical RPG', '3D', 'Cartoony', 'Old School', 'Investigation', 'Historical', 'Lore-Rich', 'War', 'Military', 'Combat']
nan ['[', ']']
nan ['Adventure', 'Exploration', 'Puzzle', '2D', 'Hidden Object', 'Colorful', 'Cute', 'Indie', 'Relaxing', 'Casual', 'Family Friendly', 'Controller', 'Singleplayer']
nan ['Strategy', 'Wargame', 'Historical', 'Turn-Based Tactics', 'Singleplayer', 'War']
nan ['Adventure', 'RPG', 'Point & Click', 'Puzzle', 'Turn-Based Strategy', 'Choose Your Own Adventure', 'Comedy', '2D', 'Funny', 'Lovecraftian', 'CRPG', 'Investigation', 'Character Customization', 'Choices Matter', 'Controller', 'Multiple Endings', 'Open World', 'Story Rich', 'Turn-Based Combat', 'Singleplayer']
nan ['Casual', 'Retro', 'Singleplayer', 'Multiplayer', 'RP

In [226]:
for i, row in df.iterrows():
    if isinstance(row['tags'], float) and isinstance(row['new_tags'], list): 
        df.at[i, 'tags'] = row['new_tags']

In [227]:
df['tags'].isna().value_counts()

tags
False    5031
True       30
Name: count, dtype: int64

The `new_tags` feature is now not that useful and can be dropped

In [228]:
df.drop(columns='new_tags', inplace=True)

In [229]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5061 entries, 0 to 5060
Data columns (total 50 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   appid                     5061 non-null   int64         
 1   normalized_name           5061 non-null   object        
 2   name                      5061 non-null   object        
 3   release_date              5061 non-null   datetime64[ns]
 4   peak_ccu                  5061 non-null   float64       
 5   required_age              5061 non-null   int64         
 6   dlc_count                 5061 non-null   int64         
 7   about_the_game            5045 non-null   object        
 8   summary                   3021 non-null   object        
 9   supported_languages       5061 non-null   object        
 10  full_audio_languages      5061 non-null   object        
 11  some_reviews              2248 non-null   object        
 12  reviews_total       

In [230]:
df['screenshots'].values

array(['https://cdn.akamai.steamstatic.com/steam/apps/1026420/ss_6a8fc6b2c001cf47ee616159e0b9038ec7bc5866.1920x1080.jpg?t=1657716289,https://cdn.akamai.steamstatic.com/steam/apps/1026420/ss_df4fa138bce054ecf5b5364f425a5883f3d2a14d.1920x1080.jpg?t=1657716289,https://cdn.akamai.steamstatic.com/steam/apps/1026420/ss_a2b404c25b07dd62106c14e7db330e5af778564e.1920x1080.jpg?t=1657716289,https://cdn.akamai.steamstatic.com/steam/apps/1026420/ss_68564ec1bb8081be3d2cc89e36a00694370b6f6a.1920x1080.jpg?t=1657716289,https://cdn.akamai.steamstatic.com/steam/apps/1026420/ss_d53b8acaaa4fdc3af645c644c3be2cffcd2620bf.1920x1080.jpg?t=1657716289,https://cdn.akamai.steamstatic.com/steam/apps/1026420/ss_976f0a68b1c4511162e59732014e69afb3680655.1920x1080.jpg?t=1657716289,https://cdn.akamai.steamstatic.com/steam/apps/1026420/ss_f0424f82e2023ce101e6e04530d895169a300095.1920x1080.jpg?t=1657716289,https://cdn.akamai.steamstatic.com/steam/apps/1026420/ss_dd10c156c29304af6ec37f06923e023b7a3eec57.1920x1080.jpg?t=165

In [231]:
df['screenshots'] = df['screenshots'].str.split(',').values

In [232]:
df['screenshots'].values

array([list(['https://cdn.akamai.steamstatic.com/steam/apps/1026420/ss_6a8fc6b2c001cf47ee616159e0b9038ec7bc5866.1920x1080.jpg?t=1657716289', 'https://cdn.akamai.steamstatic.com/steam/apps/1026420/ss_df4fa138bce054ecf5b5364f425a5883f3d2a14d.1920x1080.jpg?t=1657716289', 'https://cdn.akamai.steamstatic.com/steam/apps/1026420/ss_a2b404c25b07dd62106c14e7db330e5af778564e.1920x1080.jpg?t=1657716289', 'https://cdn.akamai.steamstatic.com/steam/apps/1026420/ss_68564ec1bb8081be3d2cc89e36a00694370b6f6a.1920x1080.jpg?t=1657716289', 'https://cdn.akamai.steamstatic.com/steam/apps/1026420/ss_d53b8acaaa4fdc3af645c644c3be2cffcd2620bf.1920x1080.jpg?t=1657716289', 'https://cdn.akamai.steamstatic.com/steam/apps/1026420/ss_976f0a68b1c4511162e59732014e69afb3680655.1920x1080.jpg?t=1657716289', 'https://cdn.akamai.steamstatic.com/steam/apps/1026420/ss_f0424f82e2023ce101e6e04530d895169a300095.1920x1080.jpg?t=1657716289', 'https://cdn.akamai.steamstatic.com/steam/apps/1026420/ss_dd10c156c29304af6ec37f06923e023b7

In [233]:
df['movies'].values

array(['http://cdn.akamai.steamstatic.com/steam/apps/256763461/movie_max.mp4?t=1573634285,http://cdn.akamai.steamstatic.com/steam/apps/256743901/movie_max.mp4?t=1551209280',
       nan,
       'http://cdn.akamai.steamstatic.com/steam/apps/2029553/movie_max.mp4?t=1490862566,http://cdn.akamai.steamstatic.com/steam/apps/2030185/movie_max.mp4?t=1490862575,http://cdn.akamai.steamstatic.com/steam/apps/2028438/movie_max.mp4?t=1490862584,http://cdn.akamai.steamstatic.com/steam/apps/2030143/movie_max.mp4?t=1493809128,http://cdn.akamai.steamstatic.com/steam/apps/2030092/movie_max.mp4?t=1493809135',
       ...,
       'http://video.akamai.steamstatic.com/store_trailers/257043794/movie_max.mp4?t=1723129231,http://video.akamai.steamstatic.com/store_trailers/257028248/movie_max.mp4?t=1717518619,http://video.akamai.steamstatic.com/store_trailers/257016381/movie_max.mp4?t=1713363951,http://video.akamai.steamstatic.com/store_trailers/257023127/movie_max.mp4?t=1716068557,http://video.akamai.steamstatic.

In [234]:
df['movies'] = df['movies'].str.split(',').values

In [235]:
df['movies'].values

array([list(['http://cdn.akamai.steamstatic.com/steam/apps/256763461/movie_max.mp4?t=1573634285', 'http://cdn.akamai.steamstatic.com/steam/apps/256743901/movie_max.mp4?t=1551209280']),
       nan,
       list(['http://cdn.akamai.steamstatic.com/steam/apps/2029553/movie_max.mp4?t=1490862566', 'http://cdn.akamai.steamstatic.com/steam/apps/2030185/movie_max.mp4?t=1490862575', 'http://cdn.akamai.steamstatic.com/steam/apps/2028438/movie_max.mp4?t=1490862584', 'http://cdn.akamai.steamstatic.com/steam/apps/2030143/movie_max.mp4?t=1493809128', 'http://cdn.akamai.steamstatic.com/steam/apps/2030092/movie_max.mp4?t=1493809135']),
       ...,
       list(['http://video.akamai.steamstatic.com/store_trailers/257043794/movie_max.mp4?t=1723129231', 'http://video.akamai.steamstatic.com/store_trailers/257028248/movie_max.mp4?t=1717518619', 'http://video.akamai.steamstatic.com/store_trailers/257016381/movie_max.mp4?t=1713363951', 'http://video.akamai.steamstatic.com/store_trailers/257023127/movie_max.mp4

In [236]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5061 entries, 0 to 5060
Data columns (total 50 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   appid                     5061 non-null   int64         
 1   normalized_name           5061 non-null   object        
 2   name                      5061 non-null   object        
 3   release_date              5061 non-null   datetime64[ns]
 4   peak_ccu                  5061 non-null   float64       
 5   required_age              5061 non-null   int64         
 6   dlc_count                 5061 non-null   int64         
 7   about_the_game            5045 non-null   object        
 8   summary                   3021 non-null   object        
 9   supported_languages       5061 non-null   object        
 10  full_audio_languages      5061 non-null   object        
 11  some_reviews              2248 non-null   object        
 12  reviews_total       

In [237]:
df['plays']

0        NaN
1         34
2        NaN
3       7.8K
4       5.1K
        ... 
5056       0
5057     NaN
5058     NaN
5059     NaN
5060     NaN
Name: plays, Length: 5061, dtype: object

In [238]:
df['plays'] = df['plays'].apply(dealing_with_Ks)

In [239]:
df['plays']

0          NaN
1         34.0
2          NaN
3       7800.0
4       5100.0
         ...  
5056       0.0
5057       NaN
5058       NaN
5059       NaN
5060       NaN
Name: plays, Length: 5061, dtype: float64

In [240]:
df['plays'].isna().value_counts()

plays
False    3024
True     2037
Name: count, dtype: int64

In [241]:
(df['plays'] == 0).value_counts()

plays
False    4999
True       62
Name: count, dtype: int64

In [242]:
df['playing']

0       NaN
1         1
2       NaN
3       203
4       191
       ... 
5056      0
5057    NaN
5058    NaN
5059    NaN
5060    NaN
Name: playing, Length: 5061, dtype: object

In [243]:
df['playing'] = df['playing'].apply(dealing_with_Ks)

In [244]:
df['playing'].isna().value_counts()

playing
False    3024
True     2037
Name: count, dtype: int64

In [245]:
(df['playing'] == 0).value_counts()

playing
False    4449
True      612
Name: count, dtype: int64

In [246]:
df['backlogs']

0        NaN
1         58
2        NaN
3       1.5K
4        461
        ... 
5056       0
5057     NaN
5058     NaN
5059     NaN
5060     NaN
Name: backlogs, Length: 5061, dtype: object

In [247]:
df['backlogs'] = df['backlogs'].apply(dealing_with_Ks)

In [248]:
df['backlogs']

0          NaN
1         58.0
2          NaN
3       1500.0
4        461.0
         ...  
5056       0.0
5057       NaN
5058       NaN
5059       NaN
5060       NaN
Name: backlogs, Length: 5061, dtype: float64

In [249]:
df['backlogs'].isna().value_counts()

backlogs
False    3024
True     2037
Name: count, dtype: int64

In [250]:
(df['backlogs'] == 0).value_counts()

backlogs
False    5025
True       36
Name: count, dtype: int64

In [251]:
df['wishlist']

0       NaN
1         5
2       NaN
3       542
4       233
       ... 
5056      0
5057    NaN
5058    NaN
5059    NaN
5060    NaN
Name: wishlist, Length: 5061, dtype: object

In [252]:
df['wishlist'] = df['wishlist'].apply(dealing_with_Ks)

In [253]:
df['wishlist'].isna().value_counts()

wishlist
False    3024
True     2037
Name: count, dtype: int64

In [254]:
(df['wishlist'] == 0).value_counts()

wishlist
False    4971
True       90
Name: count, dtype: int64

In [255]:
df['lists']

0       NaN
1        26
2       NaN
3       649
4       487
       ... 
5056      2
5057    NaN
5058    NaN
5059    NaN
5060    NaN
Name: lists, Length: 5061, dtype: object

In [256]:
df['lists'] = df['lists'].apply(dealing_with_Ks)

In [257]:
df['lists'].isna().value_counts()

lists
False    3024
True     2037
Name: count, dtype: int64

In [258]:
(df['lists'] == 0).value_counts()

lists
False    4994
True       67
Name: count, dtype: int64

In [259]:
backloggd = pd.read_csv('backloggd_contatori.csv')

In [260]:
df = df.merge(backloggd, left_on='appid', right_on='appid', how='left')

In [261]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5061 entries, 0 to 5060
Data columns (total 58 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   appid                     5061 non-null   int64         
 1   normalized_name           5061 non-null   object        
 2   name                      5061 non-null   object        
 3   release_date              5061 non-null   datetime64[ns]
 4   peak_ccu                  5061 non-null   float64       
 5   required_age              5061 non-null   int64         
 6   dlc_count                 5061 non-null   int64         
 7   about_the_game            5045 non-null   object        
 8   summary                   3021 non-null   object        
 9   supported_languages       5061 non-null   object        
 10  full_audio_languages      5061 non-null   object        
 11  some_reviews              2248 non-null   object        
 12  reviews_total       

In [262]:
df.drop(columns=['plays','playing','backlogs','wishlist','lists','game'],inplace=True)
df.rename(columns={'Plays':'plays_backloggd', 'Playing':'playing_backloggd', 'Backlogs':'backlogs_backloggd','Wishlist':'wishlist_backloggd',
                   'Lists':'lists_backloggd','Reviews':'reviews_backloggd','Likes':'likes_backloggd'}, inplace=True)

In [263]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5061 entries, 0 to 5060
Data columns (total 52 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   appid                     5061 non-null   int64         
 1   normalized_name           5061 non-null   object        
 2   name                      5061 non-null   object        
 3   release_date              5061 non-null   datetime64[ns]
 4   peak_ccu                  5061 non-null   float64       
 5   required_age              5061 non-null   int64         
 6   dlc_count                 5061 non-null   int64         
 7   about_the_game            5045 non-null   object        
 8   summary                   3021 non-null   object        
 9   supported_languages       5061 non-null   object        
 10  full_audio_languages      5061 non-null   object        
 11  some_reviews              2248 non-null   object        
 12  reviews_total       

In [264]:
df['steam_page'].values

array(['https://store.steampowered.com/app/1026420',
       'https://store.steampowered.com/app/22670',
       'https://store.steampowered.com/app/231330', ..., nan, nan, nan],
      dtype=object)

In [265]:
def steam_url_getter(appid : str) -> str:
    return 'https://store.steampowered.com/app/' + str(appid)

In [266]:
for i, row in df.iterrows():
    if pd.isna(row['steam_page']):
        df.at[i, 'steam_page'] = steam_url_getter(row['appid'])

In [267]:
df['steam_page'].values

array(['https://store.steampowered.com/app/1026420',
       'https://store.steampowered.com/app/22670',
       'https://store.steampowered.com/app/231330', ...,
       'https://store.steampowered.com/app/2396240',
       'https://store.steampowered.com/app/2305840',
       'https://store.steampowered.com/app/2366980'], dtype=object)

In [268]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5061 entries, 0 to 5060
Data columns (total 52 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   appid                     5061 non-null   int64         
 1   normalized_name           5061 non-null   object        
 2   name                      5061 non-null   object        
 3   release_date              5061 non-null   datetime64[ns]
 4   peak_ccu                  5061 non-null   float64       
 5   required_age              5061 non-null   int64         
 6   dlc_count                 5061 non-null   int64         
 7   about_the_game            5045 non-null   object        
 8   summary                   3021 non-null   object        
 9   supported_languages       5061 non-null   object        
 10  full_audio_languages      5061 non-null   object        
 11  some_reviews              2248 non-null   object        
 12  reviews_total       

In [269]:
df['platforms']

0                                                     NaN
1       ['Windows PC', 'Xbox 360', 'Linux', 'PlayStati...
2                                                     NaN
3       ['Windows PC', 'PlayStation 4', 'Xbox One', 'G...
4               ['Windows PC', 'Xbox One', 'Xbox Series']
                              ...                        
5056                                       ['Windows PC']
5057                                                  NaN
5058                                                  NaN
5059                                                  NaN
5060                                                  NaN
Name: platforms, Length: 5061, dtype: object

In [270]:
for i, row in df.iterrows():
    if isinstance(row['platforms'], str):
        df.at[i, 'platforms'] = eval(row['platforms'])

In [271]:
df['platforms'].values

array([nan, list(['Windows PC', 'Xbox 360', 'Linux', 'PlayStation 3']),
       nan, ..., nan, nan, nan], dtype=object)

In [272]:
temp_list = list()
for sublist in df['platforms'].values:
    if isinstance(sublist, list):
        for item in sublist:
            temp_list.append(item)

unique_platforms = list(set(temp_list))

In [273]:
unique_platforms

['Mobile',
 'Daydream',
 'PC DOS',
 'Game Boy Color',
 'Family Computer (FAMICOM)',
 'Sega Game Gear',
 'NES',
 'Neo Geo CD',
 'Philips CD-i',
 'SNES',
 'Windows Phone',
 'Atari Jaguar CD',
 'Neo Geo AES',
 'Sega Mega Drive/Genesis',
 'Nintendo GameCube',
 'New Nintendo 3DS',
 'Game Boy Advance',
 'Google Stadia',
 'Game Boy',
 'Playstation VR2',
 'Dreamcast',
 'Oculus Quest 2',
 'PlayStation Network',
 'Oculus Rift',
 'Nintendo Switch',
 'Xbox Series',
 'BlackBerry OS',
 'Gear VR',
 'Xbox One',
 'PlayStation Vita',
 'Mac',
 'Wii U',
 'Nintendo DS',
 'Zeebo',
 'Oculus Quest',
 'Tapwave Zodiac',
 'Linux',
 'PlayStation 2',
 'SteamVR',
 'Sega Saturn',
 'Amazon Fire TV',
 'PlayStation 3',
 'PlayStation 4',
 'Oculus VR',
 'Arcade',
 'Web browser',
 'Sega CD',
 'PlayStation VR',
 '3DO Interactive Multiplayer',
 'Xbox 360',
 'Xbox',
 'Nintendo DSi',
 'Android',
 'Satellaview',
 'Atari Jaguar',
 'Atari ST/STE',
 'Super Famicom',
 'iOS',
 'Windows PC',
 'Amiga CD32',
 'Nintendo 64',
 'Neo Geo 

In [274]:
for i, row in df.iterrows():
    if isinstance(row['platforms_2'], str):
        df.at[i, 'platforms_2'] = eval(row['platforms_2'])

In [275]:
df['platforms_2'].values

array([list(['XOne', 'NS', 'PC', 'PS4']), list(['PSN', 'XBL', 'PC']),
       list(['X360', 'PC', 'PS3']), ..., nan, nan, nan], dtype=object)

In [276]:
temp_list = list()
for sublist in df['platforms_2'].values:
    if isinstance(sublist, list):
        for item in sublist:
            temp_list.append(item)

unique_platforms_2 = list(set(temp_list))

In [277]:
unique_platforms_2

['GBA',
 'OSX',
 'PS5',
 'OR',
 'NES',
 'GBC',
 'ACPC',
 'And',
 'SNES',
 'PS2',
 'N64',
 '2600',
 'X360',
 'ApII',
 'Series',
 'SCD',
 'PS3',
 'WiiU',
 'GC',
 'PSN',
 'XS',
 'GEN',
 'DC',
 'Linux',
 'SAT',
 'GG',
 'Mob',
 'AJ',
 'DS',
 'GB',
 'XBL',
 'NGage',
 'NG',
 'MSX',
 'XB',
 'PC',
 'WS',
 'WW',
 'MS',
 '3DS',
 'TG16',
 'PSV',
 'DSiW',
 'iOS',
 'PCFX',
 'WinP',
 'Lynx',
 'C64',
 'Ouya',
 'PS',
 'PS4',
 'Wii',
 'NS',
 'VC',
 '3DO',
 'PSP',
 'S32X',
 'XOne',
 'Arc']

In [278]:
sorted(unique_platforms)

['3DO Interactive Multiplayer',
 'Amazon Fire TV',
 'Amiga',
 'Amiga CD32',
 'Android',
 'Arcade',
 'Atari Jaguar',
 'Atari Jaguar CD',
 'Atari ST/STE',
 'BlackBerry OS',
 'Daydream',
 'Dreamcast',
 'Family Computer (FAMICOM)',
 'Game Boy',
 'Game Boy Advance',
 'Game Boy Color',
 'Gear VR',
 'Google Stadia',
 'Linux',
 'Mac',
 'Mobile',
 'N-Gage',
 'NES',
 'Neo Geo AES',
 'Neo Geo CD',
 'Neo Geo MVS',
 'New Nintendo 3DS',
 'Nintendo 3DS',
 'Nintendo 64',
 'Nintendo DS',
 'Nintendo DSi',
 'Nintendo GameCube',
 'Nintendo Switch',
 'Oculus Quest',
 'Oculus Quest 2',
 'Oculus Rift',
 'Oculus VR',
 'OnLive Game System',
 'Ouya',
 'PC DOS',
 'Philips CD-i',
 'PlayStation',
 'PlayStation 2',
 'PlayStation 3',
 'PlayStation 4',
 'PlayStation 5',
 'PlayStation Network',
 'PlayStation Portable',
 'PlayStation VR',
 'PlayStation Vita',
 'Playstation VR2',
 'SNES',
 'Satellaview',
 'Sega CD',
 'Sega Game Gear',
 'Sega Mega Drive/Genesis',
 'Sega Saturn',
 'SteamVR',
 'Super Famicom',
 'Tapwave Zo

In [279]:
platforms_2_mapping = {
    'PS2': 'PlayStation 2',
    'PS5': 'PlayStation 5',
    'WiiU': 'Wii U',
    'X360': 'Xbox 360',
    'N64': 'Nintendo 64',
    'Series': 'Xbox Series',
    'GB': 'Game Boy',
    'WW': 'WiiWare', # Check V
    'GC': 'Nintendo GameCube',
    'NG': 'Neo Geo AES', # Check V
    'DC': 'Dreamcast',
    'S32X': 'Sega 32X', # Check X
    'GEN': 'Sega Mega Drive/Genesis',
    'XS': 'Xbox Series',
    'TG16': 'TurboGrafx-16', # Check X
    'VC': 'Virtual Console', # Check V
    'Linux': 'Linux',
    'OSX': 'Mac',
    'SAT': 'Sega Saturn',
    'GBC': 'Game Boy Color',
    'NES': 'NES',
    'SCD': 'Sega CD',
    'MS': 'Sega Master System', # Check V
    'Lynx': 'Atari Lynx', # Check V
    'ACPC': 'Amstrad CPC', # Check V
    'PS3': 'PlayStation 3',
    'PSP': 'PlayStation Portable',
    'PSV': 'PlayStation Vita',
    'NGage': 'N-Gage',
    'GBA': 'Game Boy Advance',
    'NS': 'Nintendo Switch',
    'XBL': 'Xbox Live', # Check V
    'XOne': 'Xbox One',
    'Arc': 'Arcade',
    'MSX': 'MSX', # Check V
    'SNES': 'SNES',
    'XB': 'Xbox',
    '3DO': '3DO Interactive Multiplayer',
    '3DS': 'Nintendo 3DS',
    'PCFX': 'NEC PC-FX', # Check X
    'Ouya': 'Ouya',
    'PC': 'Windows PC',
    'ApII': 'Apple II', # Check V
    'DS': 'Nintendo DS',
    'Wii': 'Wii',
    'iOS': 'iOS',
    'WinP': 'Windows Phone',
    'PS': 'PlayStation',
    'And': 'Android',
    'Mob': 'Mobile',
    '2600': 'Atari 2600', # Check V
    'DSiW': 'Nintendo DSi',
    'PSN': 'PlayStation Network',
    'C64': 'Commodore 64', # Check V
    'GG': 'Sega Game Gear',
    'PS4': 'PlayStation 4',
    'OR': 'Oculus Rift',
    'AJ': 'Atari Jaguar', # Check V
    'WS': 'WonderSwan'
}


In [280]:
platform_2_code = 'AJ'
for i, row in df.iterrows():
    if isinstance(row['platforms_2'], list) and platform_2_code in row['platforms_2']:
        print(row['appid'], row['name'], row['platforms'], row['platforms_2'])

227380 Dragon's Lair ['Android', 'Mac', 'Xbox 360', 'iOS', 'PlayStation 3', 'Atari ST/STE', 'Arcade', 'Sega CD', 'Amiga', 'Game Boy Color', 'Philips CD-i', 'PC DOS', '3DO Interactive Multiplayer', 'Atari Jaguar', 'Atari Jaguar CD'] ['DS', 'PSN', 'NES', 'SCD', 'DSiW', 'AJ', 'SNES', '3DO']
379720 DOOM ['Windows PC', 'PlayStation 4', 'Xbox One'] ['PS', 'GBA', 'PC', 'XBL', 'SAT', 'AJ', 'SNES', '3DO', 'PS4', 'XOne', 'And', 'NS', 'iOS']
2270 Wolfenstein 3D ['Windows PC', 'Mac', 'Xbox 360', 'iOS', 'PlayStation 3', 'Super Famicom', 'SNES', 'PC DOS', 'Web browser', 'Xbox One', 'Game Boy Advance', '3DO Interactive Multiplayer', 'Atari Jaguar'] ['GBA', 'SNES', 'XBL', 'PC', '3DO', 'AJ', 'PSN']


In [281]:
new_platforms_2_list = list()
for i, row in df.iterrows():
    if isinstance(row['platforms_2'], list):
        temp_list = list()
        for console in row['platforms_2']:
            temp_list.append(platforms_2_mapping.get(console, console))
        new_platforms_2_list.append(temp_list)
    else:
        new_platforms_2_list.append(row['platforms_2'])

In [282]:
new_platforms_2_list

[['Xbox One', 'Nintendo Switch', 'Windows PC', 'PlayStation 4'],
 ['PlayStation Network', 'Xbox Live', 'Windows PC'],
 ['Xbox 360', 'Windows PC', 'PlayStation 3'],
 ['PlayStation 4', 'Xbox One', 'Windows PC'],
 ['Xbox One', 'Windows PC', 'Xbox Series'],
 ['PlayStation 2',
  'Xbox',
  'Game Boy Advance',
  'Windows PC',
  'Xbox Series',
  'Xbox Live'],
 nan,
 ['Windows PC'],
 ['Nintendo 3DS',
  'PlayStation 4',
  'PlayStation Vita',
  'PlayStation 3',
  'Wii U',
  'Nintendo Switch',
  'Xbox One'],
 ['PlayStation 4', 'Xbox One', 'Windows PC'],
 ['Windows PC'],
 nan,
 ['Xbox 360',
  'PlayStation 3',
  'Windows PC',
  'Nintendo DS',
  'PlayStation Network',
  'Xbox Live'],
 ['Xbox Live'],
 ['PlayStation Network', 'Windows PC', 'PlayStation 4'],
 nan,
 ['PlayStation Vita', 'PlayStation 3'],
 ['Windows PC'],
 ['Nintendo Switch', 'PlayStation 4', 'Xbox One', 'Windows PC'],
 ['Nintendo Switch',
  'Windows PC',
  'PlayStation 4',
  'PlayStation 5',
  'Xbox One',
  'Xbox Series'],
 ['PlayStation

In [283]:
count = 0
for elem in new_platforms_2_list:
    if isinstance(elem, list):
        for console in elem:
            if console in ['Sega 32X', 'TurboGrafx-16', 'NEC PC-FX']:
                count += 1

print(count)

3


In [284]:
for elem in new_platforms_2_list:
    if isinstance(elem, list):
        for console in elem:
            if console in ['Sega 32X', 'TurboGrafx-16', 'NEC PC-FX']:
                elem.remove(console)

In [285]:
count = 0
for elem in new_platforms_2_list:
    if isinstance(elem, list):
        for console in elem:
            if console in ['Sega 32X', 'TurboGrafx-16', 'NEC PC-FX']:
                count += 1

print(count)

0


In [286]:
df['platforms_2'] = new_platforms_2_list

In [287]:
df[['platforms', 'platforms_2']]

,platforms,platforms_2
0,NaN,"[Xbox One, Nintendo Switch, Windows PC, PlaySt..."
1,"[Windows PC, Xbox 360, Linux, PlayStation 3]","[PlayStation Network, Xbox Live, Windows PC]"
2,NaN,"[Xbox 360, Windows PC, PlayStation 3]"
3,"[Windows PC, PlayStation 4, Xbox One, Google S...","[PlayStation 4, Xbox One, Windows PC]"
4,"[Windows PC, Xbox One, Xbox Series]","[Xbox One, Windows PC, Xbox Series]"
...,...,...
5056,[Windows PC],NaN
5057,NaN,NaN
5058,NaN,NaN
5059,NaN,NaN


The next step is to fill the NaN values of `platforms` with the non null values of `platforms_2`. When both have non missing value we have to compare the two lists

In [288]:
((df['platforms'].isna()) & (df['platforms_2'].notna())).value_counts()

False    3997
True     1064
Name: count, dtype: int64

In [289]:
df['platforms'].isna().value_counts()

platforms
False    3024
True     2037
Name: count, dtype: int64

In [290]:
df['platforms_2'].isna().value_counts()

platforms_2
False    3384
True     1677
Name: count, dtype: int64

In [291]:
df['platforms'] = df['platforms'].fillna(df['platforms_2'])

In [292]:
df['platforms'].isna().value_counts()

platforms
False    4088
True      973
Name: count, dtype: int64

In [293]:
df['platforms_2'].values

array([list(['Xbox One', 'Nintendo Switch', 'Windows PC', 'PlayStation 4']),
       list(['PlayStation Network', 'Xbox Live', 'Windows PC']),
       list(['Xbox 360', 'Windows PC', 'PlayStation 3']), ..., nan, nan,
       nan], dtype=object)

In [294]:
for elem, elem_2 in zip(df['platforms'].values, df['platforms_2'].values):
    if isinstance(elem, list) and isinstance(elem_2, list):
        print(elem, elem_2)

['Xbox One', 'Nintendo Switch', 'Windows PC', 'PlayStation 4'] ['Xbox One', 'Nintendo Switch', 'Windows PC', 'PlayStation 4']
['Windows PC', 'Xbox 360', 'Linux', 'PlayStation 3'] ['PlayStation Network', 'Xbox Live', 'Windows PC']
['Xbox 360', 'Windows PC', 'PlayStation 3'] ['Xbox 360', 'Windows PC', 'PlayStation 3']
['Windows PC', 'PlayStation 4', 'Xbox One', 'Google Stadia'] ['PlayStation 4', 'Xbox One', 'Windows PC']
['Windows PC', 'Xbox One', 'Xbox Series'] ['Xbox One', 'Windows PC', 'Xbox Series']
['Windows PC', 'Android', 'Mac', 'Xbox', 'Xbox 360', 'PlayStation 4', 'iOS', 'PlayStation 3', 'PlayStation 2'] ['PlayStation 2', 'Xbox', 'Game Boy Advance', 'Windows PC', 'Xbox Series', 'Xbox Live']
['Windows PC', 'Mac', 'Linux'] ['Windows PC']
['Windows PC', 'Wii U', 'PlayStation 4', 'PlayStation 3', 'Nintendo 3DS', 'Xbox One', 'PlayStation Vita', 'Nintendo Switch'] ['Nintendo 3DS', 'PlayStation 4', 'PlayStation Vita', 'PlayStation 3', 'Wii U', 'Nintendo Switch', 'Xbox One']
['Windows PC

In [295]:
new_platforms_1_2_list = list()
for elem, elem_2 in zip(df['platforms'].values, df['platforms_2'].values):
    if isinstance(elem, list) and isinstance(elem_2, list):
        new_platforms_1_2_list.append(list(set(elem + elem_2)))
    else:
        new_platforms_1_2_list.append(elem)

In [296]:
new_platforms_1_2_list

[['Xbox One', 'PlayStation 4', 'Nintendo Switch', 'Windows PC'],
 ['PlayStation Network',
  'PlayStation 3',
  'Xbox Live',
  'Xbox 360',
  'Linux',
  'Windows PC'],
 ['PlayStation 3', 'Xbox 360', 'Windows PC'],
 ['Google Stadia', 'Xbox One', 'PlayStation 4', 'Windows PC'],
 ['Xbox One', 'Xbox Series', 'Windows PC'],
 ['PlayStation 3',
  'PlayStation 4',
  'Xbox Series',
  'Xbox Live',
  'Xbox 360',
  'Xbox',
  'Mac',
  'Android',
  'PlayStation 2',
  'Game Boy Advance',
  'iOS',
  'Windows PC'],
 nan,
 ['Linux', 'Mac', 'Windows PC'],
 ['PlayStation 3',
  'PlayStation 4',
  'Nintendo Switch',
  'Xbox One',
  'PlayStation Vita',
  'Wii U',
  'Nintendo 3DS',
  'Windows PC'],
 ['Xbox One', 'PlayStation 4', 'Windows PC'],
 ['Oculus Quest 2',
  'Oculus VR',
  'Oculus Quest',
  'SteamVR',
  'Windows PC',
  'Playstation VR2'],
 ['Windows PC',
  'PlayStation 4',
  'Xbox One',
  'PlayStation Vita',
  'Nintendo Switch'],
 ['PlayStation 3',
  'PlayStation Network',
  'Xbox Live',
  'Xbox 360',
  

In [297]:
df['platforms'] = new_platforms_1_2_list

`platforms_2` is now useless and can be dropped

In [298]:
df.drop(columns='platforms_2', inplace=True)

In [299]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5061 entries, 0 to 5060
Data columns (total 51 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   appid                     5061 non-null   int64         
 1   normalized_name           5061 non-null   object        
 2   name                      5061 non-null   object        
 3   release_date              5061 non-null   datetime64[ns]
 4   peak_ccu                  5061 non-null   float64       
 5   required_age              5061 non-null   int64         
 6   dlc_count                 5061 non-null   int64         
 7   about_the_game            5045 non-null   object        
 8   summary                   3021 non-null   object        
 9   supported_languages       5061 non-null   object        
 10  full_audio_languages      5061 non-null   object        
 11  some_reviews              2248 non-null   object        
 12  reviews_total       

In [300]:
df[df['infos_per_platform'].notna()]['infos_per_platform'][0]

"{'XOne': {'critic_score': nan, 'total_sales': nan, 'na_sales': nan, 'jp_sales': nan, 'pal_sales': nan, 'other_sales': nan, 'release_date': '2020-10-02', 'last_update': '2020-09-11'}, 'NS': {'critic_score': nan, 'total_sales': nan, 'na_sales': nan, 'jp_sales': nan, 'pal_sales': nan, 'other_sales': nan, 'release_date': '2020-10-01', 'last_update': '2020-09-11'}, 'PC': {'critic_score': nan, 'total_sales': nan, 'na_sales': nan, 'jp_sales': nan, 'pal_sales': nan, 'other_sales': nan, 'release_date': '2019-10-02', 'last_update': '2020-09-11'}, 'PS4': {'critic_score': nan, 'total_sales': nan, 'na_sales': nan, 'jp_sales': nan, 'pal_sales': nan, 'other_sales': nan, 'release_date': '2020-09-29', 'last_update': '2020-09-11'}}"

In [301]:
info_list = list()
for elem in df['infos_per_platform'].values:
    if isinstance(elem, str):
        info_list.append(eval(elem, {"nan": np.nan}))
    else:
        info_list.append(elem)

In [302]:
df['infos_per_platform'] = info_list

In [303]:
df[df['infos_per_platform'].notna()]['infos_per_platform']

0       {'XOne': {'critic_score': nan, 'total_sales': ...
1       {'PSN': {'critic_score': 6.5, 'total_sales': n...
2       {'X360': {'critic_score': nan, 'total_sales': ...
3       {'PS4': {'critic_score': 7.9, 'total_sales': 3...
4       {'XOne': {'critic_score': 9.2, 'total_sales': ...
                              ...                        
5037    {'All': {'critic_score': nan, 'total_sales': n...
5040    {'XS': {'critic_score': nan, 'total_sales': na...
5041    {'GBA': {'critic_score': 8.7, 'total_sales': 0...
5046    {'PS2': {'critic_score': nan, 'total_sales': 1...
5052    {'XS': {'critic_score': nan, 'total_sales': na...
Name: infos_per_platform, Length: 3384, dtype: object

In [304]:
new_infos_per_platform_list = list()
for i, row in df.iterrows():
    temp_dict = dict()
    if isinstance(row['infos_per_platform'], dict):
        for console, infos in row['infos_per_platform'].items():
            new_console = platforms_2_mapping.get(console, console)
            temp_dict[new_console] = infos
        new_infos_per_platform_list.append(temp_dict)
    else:
        new_infos_per_platform_list.append(row['infos_per_platform'])

In [305]:
new_infos_per_platform_list

[{'Xbox One': {'critic_score': nan,
   'total_sales': nan,
   'na_sales': nan,
   'jp_sales': nan,
   'pal_sales': nan,
   'other_sales': nan,
   'release_date': '2020-10-02',
   'last_update': '2020-09-11'},
  'Nintendo Switch': {'critic_score': nan,
   'total_sales': nan,
   'na_sales': nan,
   'jp_sales': nan,
   'pal_sales': nan,
   'other_sales': nan,
   'release_date': '2020-10-01',
   'last_update': '2020-09-11'},
  'Windows PC': {'critic_score': nan,
   'total_sales': nan,
   'na_sales': nan,
   'jp_sales': nan,
   'pal_sales': nan,
   'other_sales': nan,
   'release_date': '2019-10-02',
   'last_update': '2020-09-11'},
  'PlayStation 4': {'critic_score': nan,
   'total_sales': nan,
   'na_sales': nan,
   'jp_sales': nan,
   'pal_sales': nan,
   'other_sales': nan,
   'release_date': '2020-09-29',
   'last_update': '2020-09-11'}},
 {'PlayStation Network': {'critic_score': 6.5,
   'total_sales': nan,
   'na_sales': nan,
   'jp_sales': nan,
   'pal_sales': nan,
   'other_sales': 

In [306]:
count = 0
for elem in new_infos_per_platform_list:
    if isinstance(elem, dict):
        for console in list(elem.keys()):
            if console in ['Sega 32X', 'TurboGrafx-16', 'NEC PC-FX']:
                count += 1
print(count)

3


In [307]:
for elem in new_infos_per_platform_list:
    if isinstance(elem, dict):
        for console in list(elem.keys()):
            if console in ['Sega 32X', 'TurboGrafx-16', 'NEC PC-FX']:
                del elem[console]

In [308]:
count = 0
for elem in new_infos_per_platform_list:
    if isinstance(elem, dict):
        for console in list(elem.keys()):
            if console in ['Sega 32X', 'TurboGrafx-16', 'NEC PC-FX']:
                count += 1
print(count)

0


In [309]:
df['infos_per_platform'] = new_infos_per_platform_list

In [310]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5061 entries, 0 to 5060
Data columns (total 51 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   appid                     5061 non-null   int64         
 1   normalized_name           5061 non-null   object        
 2   name                      5061 non-null   object        
 3   release_date              5061 non-null   datetime64[ns]
 4   peak_ccu                  5061 non-null   float64       
 5   required_age              5061 non-null   int64         
 6   dlc_count                 5061 non-null   int64         
 7   about_the_game            5045 non-null   object        
 8   summary                   3021 non-null   object        
 9   supported_languages       5061 non-null   object        
 10  full_audio_languages      5061 non-null   object        
 11  some_reviews              2248 non-null   object        
 12  reviews_total       

In [311]:
df['revenue_estimated']

0           $11 203,33
1            $3 076,92
2           $23 548,22
3        $7 708 595,02
4       $10 485 712,09
             ...      
5056               NaN
5057               NaN
5058               NaN
5059               NaN
5060               NaN
Name: revenue_estimated, Length: 5061, dtype: object

In [312]:
df['revenue_estimated'] = df['revenue_estimated'].str.replace('\xa0', '', regex=True).str.replace('$' , '').str.replace(',' , '.').values.astype(float)

In [313]:
df['revenue_estimated']

0          11203.33
1           3076.92
2          23548.22
3        7708595.02
4       10485712.09
           ...     
5056            NaN
5057            NaN
5058            NaN
5059            NaN
5060            NaN
Name: revenue_estimated, Length: 5061, dtype: float64

In [314]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5061 entries, 0 to 5060
Data columns (total 51 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   appid                     5061 non-null   int64         
 1   normalized_name           5061 non-null   object        
 2   name                      5061 non-null   object        
 3   release_date              5061 non-null   datetime64[ns]
 4   peak_ccu                  5061 non-null   float64       
 5   required_age              5061 non-null   int64         
 6   dlc_count                 5061 non-null   int64         
 7   about_the_game            5045 non-null   object        
 8   summary                   3021 non-null   object        
 9   supported_languages       5061 non-null   object        
 10  full_audio_languages      5061 non-null   object        
 11  some_reviews              2248 non-null   object        
 12  reviews_total       

In [315]:
df[['name', 'new_initialprice', 'new_discount','new_price']]

,name,new_initialprice,new_discount,new_price
0,WARSAW,23.99,0.0,0.00
1,Alien Breed 3: Descent,9.99,0.0,9.99
2,Deadfall Adventures,19.99,0.0,19.99
3,Far Cry® 5,59.99,0.0,59.99
4,Forza Horizon 4,59.99,0.0,0.00
...,...,...,...,...
5056,Deathbound,NaN,NaN,26.99
5057,F1® Manager 2024,NaN,NaN,34.99
5058,SteamWorld Heist II,NaN,NaN,29.99
5059,Cat Quest III,NaN,NaN,19.99


In [316]:
# Notation: 
# NaN = X, NotNull = V

In [317]:
((df['new_initialprice'].notna()) & (df['new_price'].notna()) & (df['new_discount'].notna())).value_counts()

True     4583
False     478
Name: count, dtype: int64

In [318]:
# Case 1: 
# new_initialprice = V | new_price = V | new_discount = V
count = 0
for i, row in df.iterrows():
    if pd.notna(row['new_initialprice']) and pd.notna(row['new_price']) and pd.notna(row['new_discount']):
        if abs(int(row['new_initialprice']*(1 - row['new_discount']/100)*100)/100 - row['new_price']) > 0.1:
            count += 1
            print(row['new_initialprice'], row['new_discount'], '------->', int(row['new_initialprice']*(1 - row['new_discount']/100)*100)/100, 'VS', row['new_price'])
print(count)

23.99 0.0 -------> 23.99 VS 0.0
59.99 0.0 -------> 59.99 VS 0.0
19.99 0.0 -------> 19.98 VS 0.0
4.99 0.0 -------> 4.99 VS 0.0
9.99 0.0 -------> 9.99 VS 0.0
19.99 0.0 -------> 19.98 VS 0.0
39.99 0.0 -------> 39.99 VS 0.0
0.99 0.0 -------> 0.99 VS 0.0
19.99 0.0 -------> 19.98 VS 0.0
14.99 0.0 -------> 14.99 VS 0.0
19.99 0.0 -------> 19.98 VS 0.0
19.99 0.0 -------> 19.98 VS 0.0
0.99 0.0 -------> 0.99 VS 0.0
29.99 0.0 -------> 29.99 VS 0.0
29.99 0.0 -------> 29.99 VS 0.0
29.99 0.0 -------> 29.99 VS 0.0
19.99 0.0 -------> 19.98 VS 0.0
19.99 0.0 -------> 19.98 VS 0.0
3.99 0.0 -------> 3.99 VS 0.0
59.99 0.0 -------> 59.99 VS 0.0
34.99 0.0 -------> 34.99 VS 0.0
49.99 0.0 -------> 49.99 VS 0.0
39.99 0.0 -------> 39.99 VS 0.0
59.99 0.0 -------> 59.99 VS 0.0
59.99 0.0 -------> 59.99 VS 0.0
34.99 0.0 -------> 34.99 VS 0.0
19.99 0.0 -------> 19.98 VS 0.0
19.99 0.0 -------> 19.98 VS 0.0
59.99 0.0 -------> 59.99 VS 0.0
39.99 0.0 -------> 39.99 VS 0.0
17.99 0.0 -------> 17.98 VS 0.0
24.99 0.0 ------->

In [319]:
((df['new_discount'] == 0) & (df['new_price'] == 0)).value_counts()

False    4821
True      240
Name: count, dtype: int64

In [320]:
df[((df['new_discount'] == 0) & (df['new_price'] == 0))]['name']

0                            WARSAW
4                   Forza Horizon 4
50                   Lost Planet® 2
71                     Rocket Arena
77                         God Mode
                   ...             
4705    Stranger Things 3: The Game
4706                    Velocity 2X
4707           Frozen Synapse Prime
4708                    Titanfall­™
4709                Darkest of Days
Name: name, Length: 240, dtype: object

The game that have both `discount` and `new_price` equal to $0$ have high chance to be not more available on Steam

In [321]:
((df['new_initialprice'] == 0) & (df['new_discount'] == 0) & (df['new_price'] == 0)).value_counts()

False    5061
Name: count, dtype: int64

In [322]:
((df['new_initialprice'].isna()) & (df['new_price'].notna()) & (df['new_discount'].notna())).value_counts()

False    5061
Name: count, dtype: int64

In [323]:
((df['new_initialprice'].notna()) & (df['new_price'].isna()) & (df['new_discount'].notna())).value_counts()

False    5061
Name: count, dtype: int64

In [324]:
((df['new_initialprice'].notna()) & (df['new_price'].notna()) & (df['new_discount'].isna())).value_counts()

False    5061
Name: count, dtype: int64

In [325]:
((df['new_initialprice'].isna()) & (df['new_price'].isna()) & (df['new_discount'].notna())).value_counts()

False    5061
Name: count, dtype: int64

In [326]:
((df['new_initialprice'].isna()) & (df['new_price'].notna()) & (df['new_discount'].isna())).value_counts()

False    4708
True      353
Name: count, dtype: int64

In [327]:
((df['new_initialprice'].isna()) & (df['new_price'].notna())).value_counts()

False    4708
True      353
Name: count, dtype: int64

In [328]:
df[(df['new_initialprice'].isna()) & (df['new_price'].notna())]['name']

45          Wasteland 2: Director's Cut
195                            One Shot
203     Tom Clancy's Rainbow Six® Siege
205                            F.E.A.R.
215            Call of Duty®: Black Ops
                     ...               
5056                         Deathbound
5057                   F1® Manager 2024
5058                SteamWorld Heist II
5059                      Cat Quest III
5060        Thank Goodness You're Here!
Name: name, Length: 353, dtype: object

In [329]:
# Case 2:
# new_initialprice = X | new_price = V | new_discount = X

for i, row in df.iterrows():
    if pd.isna(row['new_initialprice']) and pd.notna(row['new_price']):
        df.at[i, 'new_initialprice'] = row['new_price']
        df.at[i, 'new_discount'] = float(0)

In [330]:
df[['new_initialprice', 'new_discount', 'new_price']]

,new_initialprice,new_discount,new_price
0,23.99,0.0,0.00
1,9.99,0.0,9.99
2,19.99,0.0,19.99
3,59.99,0.0,59.99
4,59.99,0.0,0.00
...,...,...,...
5056,26.99,0.0,26.99
5057,34.99,0.0,34.99
5058,29.99,0.0,29.99
5059,19.99,0.0,19.99


In [331]:
((df['new_initialprice'].notna()) & (df['new_price'].isna()) & (df['new_discount'].isna())).value_counts()

False    5061
Name: count, dtype: int64

In [332]:
((df['new_initialprice'].isna()) & (df['new_price'].isna()) & (df['new_discount'].isna())).value_counts()

False    4936
True      125
Name: count, dtype: int64

In [333]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5061 entries, 0 to 5060
Data columns (total 51 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   appid                     5061 non-null   int64         
 1   normalized_name           5061 non-null   object        
 2   name                      5061 non-null   object        
 3   release_date              5061 non-null   datetime64[ns]
 4   peak_ccu                  5061 non-null   float64       
 5   required_age              5061 non-null   int64         
 6   dlc_count                 5061 non-null   int64         
 7   about_the_game            5045 non-null   object        
 8   summary                   3021 non-null   object        
 9   supported_languages       5061 non-null   object        
 10  full_audio_languages      5061 non-null   object        
 11  some_reviews              2248 non-null   object        
 12  reviews_total       

In [334]:
df = df.rename(columns={'meta_critic_score': 'metacritic_score ', 'new_positive': 'positive', 'new_negative': 'negative', 'new_min_estimated_owners': 'min_estimated_owners', 
                        'new_max_estimated_owners': 'max_estimated_owners', 'new_average_forever': 'average_forever', 'new_median_forever': 'median_forever',
                        'new_price': 'current_price', 'new_initialprice': 'initial_price', 'new_discount': 'discount_percentage'})

In [335]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5061 entries, 0 to 5060
Data columns (total 51 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   appid                  5061 non-null   int64         
 1   normalized_name        5061 non-null   object        
 2   name                   5061 non-null   object        
 3   release_date           5061 non-null   datetime64[ns]
 4   peak_ccu               5061 non-null   float64       
 5   required_age           5061 non-null   int64         
 6   dlc_count              5061 non-null   int64         
 7   about_the_game         5045 non-null   object        
 8   summary                3021 non-null   object        
 9   supported_languages    5061 non-null   object        
 10  full_audio_languages   5061 non-null   object        
 11  some_reviews           2248 non-null   object        
 12  reviews_total          5061 non-null   float64       
 13  rev

In [336]:
df.to_csv('steam_games.csv', index=False)

In [337]:
temp_df = df[['appid', 'infos_per_platform']].copy(deep=True)

In [338]:
records = []

for _, row in temp_df.iterrows():
    appid = row['appid']
    platform_data = row['infos_per_platform']

    if isinstance(platform_data, dict):
        for platform, info in platform_data.items():
            flat_row = {'appid': appid, 'platform': platform}
            flat_row.update(info)
            records.append(flat_row)
    else:
        records.append({'appid': appid, 'platform': None})

# Create the long-form DataFrame
long_temp_df = pd.DataFrame(records)

In [339]:
long_temp_df

,appid,platform,critic_score,total_sales,na_sales,jp_sales,pal_sales,other_sales,release_date,last_update
0,1026420,Xbox One,NaN,NaN,NaN,NaN,NaN,NaN,2020-10-02,2020-09-11
1,1026420,Nintendo Switch,NaN,NaN,NaN,NaN,NaN,NaN,2020-10-01,2020-09-11
2,1026420,Windows PC,NaN,NaN,NaN,NaN,NaN,NaN,2019-10-02,2020-09-11
3,1026420,PlayStation 4,NaN,NaN,NaN,NaN,NaN,NaN,2020-09-29,2020-09-11
4,22670,PlayStation Network,6.5,NaN,NaN,NaN,NaN,NaN,2011-02-22,NaN
...,...,...,...,...,...,...,...,...,...,...
12012,1277130,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12013,2591280,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12014,2396240,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12015,2305840,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [340]:
long_temp_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12017 entries, 0 to 12016
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   appid         12017 non-null  int64  
 1   platform      10340 non-null  object 
 2   critic_score  1265 non-null   float64
 3   total_sales   2361 non-null   float64
 4   na_sales      1817 non-null   float64
 5   jp_sales      722 non-null    float64
 6   pal_sales     1899 non-null   float64
 7   other_sales   2229 non-null   float64
 8   release_date  7725 non-null   object 
 9   last_update   7377 non-null   object 
dtypes: float64(6), int64(1), object(3)
memory usage: 939.0+ KB


In [341]:
long_temp_df['release_date'] = pd.to_datetime(long_temp_df['release_date'])
long_temp_df['last_update'] = pd.to_datetime(long_temp_df['last_update'])

In [342]:
long_temp_df.columns

Index(['appid', 'platform', 'critic_score', 'total_sales', 'na_sales',
       'jp_sales', 'pal_sales', 'other_sales', 'release_date', 'last_update'],
      dtype='object')

In [343]:
long_temp_df = long_temp_df.rename(columns={'critic_score' : 'critic_score_pp', 'total_sales' : 'total_sales_pp', 'na_sales' : 'na_sales_pp',
       'jp_sales' : 'jp_sales_pp', 'pal_sales' : 'pal_sales_pp', 'other_sales' : 'other_sales_pp', 'release_date' : 'release_date_pp', 'last_update' : 'last_update_pp'})

In [344]:
long_temp_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12017 entries, 0 to 12016
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   appid            12017 non-null  int64         
 1   platform         10340 non-null  object        
 2   critic_score_pp  1265 non-null   float64       
 3   total_sales_pp   2361 non-null   float64       
 4   na_sales_pp      1817 non-null   float64       
 5   jp_sales_pp      722 non-null    float64       
 6   pal_sales_pp     1899 non-null   float64       
 7   other_sales_pp   2229 non-null   float64       
 8   release_date_pp  7725 non-null   datetime64[ns]
 9   last_update_pp   7377 non-null   datetime64[ns]
dtypes: datetime64[ns](2), float64(6), int64(1), object(1)
memory usage: 939.0+ KB


In [345]:
long_df = df.merge(long_temp_df, left_on='appid', right_on='appid', how='right')

In [346]:
long_df.drop(columns='infos_per_platform', inplace=True)

In [347]:
long_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12017 entries, 0 to 12016
Data columns (total 59 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   appid                  12017 non-null  int64         
 1   normalized_name        12017 non-null  object        
 2   name                   12017 non-null  object        
 3   release_date           12017 non-null  datetime64[ns]
 4   peak_ccu               12017 non-null  float64       
 5   required_age           12017 non-null  int64         
 6   dlc_count              12017 non-null  int64         
 7   about_the_game         11978 non-null  object        
 8   summary                8400 non-null   object        
 9   supported_languages    12017 non-null  object        
 10  full_audio_languages   12017 non-null  object        
 11  some_reviews           4884 non-null   object        
 12  reviews_total          12017 non-null  float64       
 13  r

In [348]:
long_df.head()

,appid,normalized_name,name,release_date,peak_ccu,required_age,dlc_count,about_the_game,summary,supported_languages,...,likes_backloggd,platform,critic_score_pp,total_sales_pp,na_sales_pp,jp_sales_pp,pal_sales_pp,other_sales_pp,release_date_pp,last_update_pp
0,1026420,warsaw,WARSAW,2019-10-01,5.0,0,0,Use everything at your disposal to help a team...,NaN,"[English, French, German, Polish, Russian]",...,0.0,Xbox One,NaN,NaN,NaN,NaN,NaN,NaN,2020-10-02,2020-09-11
1,1026420,warsaw,WARSAW,2019-10-01,5.0,0,0,Use everything at your disposal to help a team...,NaN,"[English, French, German, Polish, Russian]",...,0.0,Nintendo Switch,NaN,NaN,NaN,NaN,NaN,NaN,2020-10-01,2020-09-11
2,1026420,warsaw,WARSAW,2019-10-01,5.0,0,0,Use everything at your disposal to help a team...,NaN,"[English, French, German, Polish, Russian]",...,0.0,Windows PC,NaN,NaN,NaN,NaN,NaN,NaN,2019-10-02,2020-09-11
3,1026420,warsaw,WARSAW,2019-10-01,5.0,0,0,Use everything at your disposal to help a team...,NaN,"[English, French, German, Polish, Russian]",...,0.0,PlayStation 4,NaN,NaN,NaN,NaN,NaN,NaN,2020-09-29,2020-09-11
4,22670,alienbreed3descent,Alien Breed 3: Descent,2010-11-01,3.0,0,0,Alien Breed™ 3: Descent is the final explosive...,Alien Breed 3: Descent is the final explosive ...,"[English, French, German, Italian, Japanese, S...",...,0.0,PlayStation Network,6.5,NaN,NaN,NaN,NaN,NaN,2011-02-22,NaT


In [349]:
long_df.to_csv('steam_games_long.csv', index=False)

In [350]:
import pandas as pd
import numpy as np
from data_type_fix import data_type_fix

In [351]:
df = pd.read_csv('steam_games.csv')
df = data_type_fix(df)

In [352]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5061 entries, 0 to 5060
Data columns (total 51 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   appid                  5061 non-null   int64         
 1   normalized_name        5061 non-null   object        
 2   name                   5061 non-null   object        
 3   release_date           5061 non-null   datetime64[ns]
 4   peak_ccu               5061 non-null   float64       
 5   required_age           5061 non-null   int64         
 6   dlc_count              5061 non-null   int64         
 7   about_the_game         5045 non-null   object        
 8   summary                3021 non-null   object        
 9   supported_languages    5061 non-null   object        
 10  full_audio_languages   5061 non-null   object        
 11  some_reviews           2248 non-null   object        
 12  reviews_total          5061 non-null   float64       
 13  rev

In [353]:
df[['appid', 'name', 'metacritic_url', 'steam_page', 'header_image']].to_csv('games_list.csv', index=False)

In [354]:
df1 = pd.read_csv('games_list.csv')
df1.head()

,appid,name,metacritic_url,steam_page,header_image
0,1026420,WARSAW,https://www.metacritic.com/game/pc/warsaw?ftag...,https://store.steampowered.com/app/1026420,https://cdn.akamai.steamstatic.com/steam/apps/...
1,22670,Alien Breed 3: Descent,https://www.metacritic.com/game/pc/alien-breed...,https://store.steampowered.com/app/22670,https://cdn.akamai.steamstatic.com/steam/apps/...
2,231330,Deadfall Adventures,https://www.metacritic.com/game/pc/deadfall-ad...,https://store.steampowered.com/app/231330,https://cdn.akamai.steamstatic.com/steam/apps/...
3,552520,Far Cry® 5,https://www.metacritic.com/game/pc/far-cry-5?f...,https://store.steampowered.com/app/552520,https://cdn.akamai.steamstatic.com/steam/apps/...
4,1293830,Forza Horizon 4,https://www.metacritic.com/game/pc/forza-horiz...,https://store.steampowered.com/app/1293830,https://cdn.akamai.steamstatic.com/steam/apps/...


In [355]:
df1['metacritic_url'].values

array(['https://www.metacritic.com/game/pc/warsaw?ftag=MCD-06-10aaa1f',
       'https://www.metacritic.com/game/pc/alien-breed-3-descent?ftag=MCD-06-10aaa1f',
       'https://www.metacritic.com/game/pc/deadfall-adventures?ftag=MCD-06-10aaa1f',
       ...,
       'https://www.metacritic.com/game/pc/steamworld-heist-ii?ftag=MCD-06-10aaa1f',
       'https://www.metacritic.com/game/pc/cat-quest-iii?ftag=MCD-06-10aaa1f',
       'https://www.metacritic.com/game/pc/thank-goodness-youre-here!?ftag=MCD-06-10aaa1f'],
      dtype=object)

In [ ]:
import requests
from bs4 import BeautifulSoup
import time
import random
import pandas as pd

# Inserisci qui i tuoi 5000 AppID
app_ids = df['appid'].values.astype(str)  # <-- Lista completa qui

# Configura la dimensione dei batch
batch_size = 100

# Genera i batch
batches = [app_ids[i:i + batch_size] for i in range(0, len(app_ids), batch_size)]

# User-Agent per sembrare un browser reale
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/114.0.0.0 Safari/537.36"
}

def get_all_time_peak(app_id):
    url = f"https://steamcharts.com/app/{app_id}"
    try:
        response = requests.get(url, headers=headers, timeout=15)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, "html.parser")
        stat_divs = soup.find_all("div", class_="app-stat")
        if len(stat_divs) >= 3:
            peak_text = stat_divs[2].find("span", class_="num").text.strip().replace(',', '')
            return int(peak_text)
        else:
            print(f"Struttura cambiata per app_id {app_id}", end = '\r')
            return None 
    except requests.exceptions.RequestException as e:
        print(f"Errore rete con app_id {app_id}: {e}", end = '\r')
        return None
    except Exception as e:
        print(f"Errore imprevisto con app_id {app_id}: {e}", end = '\r')
        return None

def run_batch(batch_index):
    if batch_index >= len(batches):
        print("Batch index fuori dal range.")
        return

    batch_app_ids = batches[batch_index]
    peak_list = []

    print(f"\nAvvio batch {batch_index + 1}/{len(batches)} con {len(batch_app_ids)} app_id...\n")
    for app_id in batch_app_ids:
        peak = get_all_time_peak(app_id)
        peak_list.append(peak)
        print(f"{app_id} → {peak}", end = '\r')
        time.sleep(random.uniform(1, 3))  # Ritardo random per evitare blocchi

    # Salvataggio su CSV
    df = pd.DataFrame({"AppID": batch_app_ids, "AllTimePeak": peak_list})
    df.to_csv(f"steam_peak_batch_{batch_index}.csv", index=False)
    print(f"\nBatch {batch_index} completato e salvato come 'steam_peak_batch_{batch_index}.csv'.")

# Esempio di uso manuale:
# run_batch(0)  # Esegui il primo batch
# run_batch(1)  # Esegui il secondo quando vuoi

'\nimport requests\nfrom bs4 import BeautifulSoup\nimport time\nimport random\nimport pandas as pd\n\n# Inserisci qui i tuoi 5000 AppID\napp_ids = df[\'appid\'].values.astype(str)  # <-- Lista completa qui\n\n# Configura la dimensione dei batch\nbatch_size = 100\n\n# Genera i batch\nbatches = [app_ids[i:i + batch_size] for i in range(0, len(app_ids), batch_size)]\n\n# User-Agent per sembrare un browser reale\nheaders = {\n    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "\n                  "AppleWebKit/537.36 (KHTML, like Gecko) "\n                  "Chrome/114.0.0.0 Safari/537.36"\n}\n\ndef get_all_time_peak(app_id):\n    url = f"https://steamcharts.com/app/{app_id}"\n    try:\n        response = requests.get(url, headers=headers, timeout=15)\n        response.raise_for_status()\n        soup = BeautifulSoup(response.text, "html.parser")\n        stat_divs = soup.find_all("div", class_="app-stat")\n        if len(stat_divs) >= 3:\n            peak_text = stat_divs[2].find

In [357]:
'''
for i in range(41,50+1):
    run_batch(i)
    time.sleep(30)
'''

'\nfor i in range(41,50+1):\n    run_batch(i)\n    time.sleep(30)\n'

In [ ]:
import glob

# Trova tutti i CSV che seguono il pattern dei batch
csv_files = glob.glob("steam_peak_batch_*.csv")

# Legge e concatena tutti i CSV
df_list = [pd.read_csv(file) for file in csv_files]
df_all = pd.concat(df_list, ignore_index=True)

# Salva il risultato finale
df_all.to_csv("steam_peak_all.csv", index=False)

print(f"Unito {len(csv_files)} file. Totale AppID: {len(df_all)}")

'\nimport glob\n\n# Trova tutti i CSV che seguono il pattern dei batch\ncsv_files = glob.glob("steam_peak_batch_*.csv")\n\n# Legge e concatena tutti i CSV\ndf_list = [pd.read_csv(file) for file in csv_files]\ndf_all = pd.concat(df_list, ignore_index=True)\n\n# Salva il risultato finale\ndf_all.to_csv("steam_peak_all.csv", index=False)\n\nprint(f"Unito {len(csv_files)} file. Totale AppID: {len(df_all)}")\n'

In [359]:
df_all = pd.read_csv('steam_peak_all.csv')

In [360]:
df = df.merge(df_all, how='left', left_on='appid', right_on='AppID')

In [361]:
df.drop(columns='AppID', inplace = True)

In [362]:
col = df.pop('AllTimePeak')
df.insert(5, 'AllTimePeak', col)

In [363]:
df.rename(columns={'AllTimePeak': 'all_time_peak_ccu', 'peak_ccu': '24h_peak_ccu'}, inplace=True)

In [364]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5061 entries, 0 to 5060
Data columns (total 52 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   appid                  5061 non-null   int64         
 1   normalized_name        5061 non-null   object        
 2   name                   5061 non-null   object        
 3   release_date           5061 non-null   datetime64[ns]
 4   24h_peak_ccu           5061 non-null   float64       
 5   all_time_peak_ccu      3386 non-null   float64       
 6   required_age           5061 non-null   int64         
 7   dlc_count              5061 non-null   int64         
 8   about_the_game         5045 non-null   object        
 9   summary                3021 non-null   object        
 10  supported_languages    5061 non-null   object        
 11  full_audio_languages   5061 non-null   object        
 12  some_reviews           2248 non-null   object        
 13  rev

In [365]:
df.to_csv('steam_games.csv', index=False)